In [2]:
# Cell 1: Setup and Core Imports

# --- Aggressive cleanup of existing PyTorch/Torchvision installations ---
# This helps resolve issues like "partially initialized module" or "operator does not exist"
!pip cache purge -q # Clear pip cache
# Explicitly uninstall torch and torchaudio. We will avoid torchvision as it's causing issues.
!pip uninstall torch torchaudio -y -q # Force uninstall existing packages

# Install necessary dependencies (quietly)
# We will target PyTorch 2.2.0 with CUDA 12.1, common on Kaggle.
# We'll let pip resolve numpy version to avoid conflicts.
# IMPORTANT: Removed torchvision from this install line as it's the source of the problem.
!pip install torch==2.2.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu121 --no-cache-dir -q
!pip install nibabel matplotlib tqdm scikit-learn optuna --quiet

# Core Imports
import os
import nibabel as nib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler
# Removed: from torchvision import transforms # This line was causing the error
from typing import Dict, List, Tuple, Optional, Any
from tqdm.notebook import tqdm # Use tqdm.notebook for better display in Jupyter/Kaggle
import random
import warnings
import time
from pathlib import Path
import logging
import pickle
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
from skimage import measure # For 3D rendering in MedicalImageVisualizer
from scipy.spatial.distance import directed_hausdorff
from scipy.ndimage import binary_erosion, binary_dilation, generate_binary_structure
from scipy.ndimage import rotate # Added for data augmentation in BraTSDataset
from sklearn.metrics import confusion_matrix

# For TensorBoard logging (needed by Trainer)
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    print("TensorBoard not found. Please install with: pip install tensorboard")
    # Provide a dummy SummaryWriter if not installed to prevent errors
    class SummaryWriter:
        def __init__(self, log_dir): pass
        def add_scalar(self, tag, scalar_value, global_step=None): pass
        def add_scalars(self, main_tag, tag_scalar_dict, global_step=None): pass
        def close(self): pass


# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set device for PyTorch operations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

# Configure basic logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("Cell 1 executed: Setup and Core Imports")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.3/757.3 MB 287.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 294.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.2.0+cu121 which is incompatible.
Using device: cuda
GPU Name: Tesla T4
Cell 1 executed: Setup and Core Imports


In [3]:
# Cell 2: Utility Functions

class MedicalImageUtils:
    """
    A collection of static utility methods for medical image processing.
    """

    @staticmethod
    def load_nifti(path: str) -> np.ndarray:
        """Loads a NIfTI file and returns its data as a numpy array."""
        return nib.load(path).get_fdata()

    @staticmethod
    def save_nifti(data: np.ndarray, path: str, affine: Optional[np.ndarray] = None) -> None:
        """Saves a numpy array as a NIfTI file."""
        if affine is None:
            affine = np.eye(4) # Default affine if none provided
        nifti_img = nib.Nifti1Image(data, affine)
        nib.save(nifti_img, path)

    @staticmethod
    def normalize_image(image: np.ndarray) -> np.ndarray:
        """Min-max normalizes a 3D image to the range [0, 1]."""
        min_val = np.min(image)
        max_val = np.max(image)
        if max_val == min_val: # Handle cases of uniform images
            return np.zeros_like(image, dtype=np.float32)
        return ((image - min_val) / (max_val - min_val)).astype(np.float32)

    @staticmethod
    def crop_or_pad_3d(image: np.ndarray, target_size: Tuple[int, int, int]) -> np.ndarray:
        """
        Crops or pads a 3D image to a target size.
        If cropping, it crops from the center. If padding, it pads symmetrically.
        """
        current_size = image.shape
        padded_image = np.zeros(target_size, dtype=image.dtype)

        # Calculate start and end indices for cropping/pasting
        slices = [slice(0, dim) for dim in target_size]
        original_slices = [slice(0, dim) for dim in current_size]

        for i in range(3): # For each dimension (depth, height, width)
            if current_size[i] > target_size[i]: # Crop
                start = (current_size[i] - target_size[i]) // 2
                original_slices[i] = slice(start, start + target_size[i])
            elif current_size[i] < target_size[i]: # Pad
                start = (target_size[i] - current_size[i]) // 2
                slices[i] = slice(start, start + current_size[i])

        padded_image[tuple(slices)] = image[tuple(original_slices)]
        return padded_image

    @staticmethod
    def get_brain_bounding_box(mask: np.ndarray, margin: int = 5) -> Tuple[slice, slice, slice]:
        """
        Calculates the bounding box around non-zero regions of a 3D mask.
        Adds a margin for context.
        """
        coords = np.argwhere(mask > 0)
        if coords.shape[0] == 0:
            # If mask is empty, return full image slices or handle as error
            return (slice(0, mask.shape[0]), slice(0, mask.shape[1]), slice(0, mask.shape[2]))

        min_coords = np.min(coords, axis=0)
        max_coords = np.max(coords, axis=0)

        d_min, h_min, w_min = min_coords
        d_max, h_max, w_max = max_coords

        # Apply margin and ensure bounds are within image dimensions
        d_min = max(0, d_min - margin)
        h_min = max(0, h_min - margin)
        w_min = max(0, w_min - margin)

        d_max = min(mask.shape[0], d_max + margin + 1) # +1 for exclusive end
        h_max = min(mask.shape[1], h_max + margin + 1)
        w_max = min(mask.shape[2], w_max + margin + 1)

        return (slice(d_min, d_max), slice(h_min, h_max), slice(w_min, w_max))

    @staticmethod
    def augment_data(image: np.ndarray, mask: np.ndarray, angle_range: int = 15) -> Tuple[np.ndarray, np.ndarray]:
        """
        Applies random 3D rotation augmentation to image and mask.
        Assumes image and mask are (D, H, W).
        """
        # Pick random angles for rotation around each axis
        angle_d = np.random.uniform(-angle_range, angle_range)
        angle_h = np.random.uniform(-angle_range, angle_range)
        angle_w = np.random.uniform(-angle_range, angle_range)

        # Rotate image and mask. Order=0 for mask (nearest neighbor to preserve labels)
        # Order=3 for image (cubic interpolation for smoother image)
        image_aug = rotate(image, angle_d, axes=(1, 2), reshape=False, order=3, mode='nearest')
        image_aug = rotate(image_aug, angle_h, axes=(0, 2), reshape=False, order=3, mode='nearest')
        image_aug = rotate(image_aug, angle_w, axes=(0, 1), reshape=False, order=3, mode='nearest')

        mask_aug = rotate(mask, angle_d, axes=(1, 2), reshape=False, order=0, mode='nearest')
        mask_aug = rotate(mask_aug, angle_h, axes=(0, 2), reshape=False, order=0, mode='nearest')
        mask_aug = rotate(mask_aug, angle_w, axes=(0, 1), reshape=False, order=0, mode='nearest')

        return image_aug.astype(image.dtype), mask_aug.astype(mask.dtype)

print("Cell 2 executed: Utility Functions")

Cell 2 executed: Utility Functions


In [4]:
# Cell 3: BraTS Dataset Class (Updated - Crucial Fix for .nii extension)

class BraTSDataset(Dataset):
    """
    Custom PyTorch Dataset for BraTS 2020 brain tumor segmentation.
    Handles loading, preprocessing, and augmentation of 3D MRI scans.
    """
    def __init__(self, data_paths: List[str], patch_size: Tuple[int, int, int], train: bool = True):
        self.data_paths = self._parse_data_paths(data_paths)
        self.patch_size = patch_size
        self.train = train
        logger.info(f"Initialized BraTSDataset with {len(self.data_paths)} subjects. Train mode: {self.train}")

    def _parse_data_paths(self, base_paths: List[str]) -> List[Dict[str, str]]:
        """
        Parses subject base paths to full paths for each modality and seg.
        CRITICAL: Uses '.nii' extension as per your provided directory structure.
        """
        parsed_paths = []
        for base_path in base_paths:
            subject_id = os.path.basename(base_path)
            paths = {
                'id': subject_id,
                'flair': os.path.join(base_path, f'{subject_id}_flair.nii'), # Changed to .nii
                't1': os.path.join(base_path, f'{subject_id}_t1.nii'),     # Changed to .nii
                't1ce': os.path.join(base_path, f'{subject_id}_t1ce.nii'), # Changed to .nii
                't2': os.path.join(base_path, f'{subject_id}_t2.nii'),     # Changed to .nii
                'seg': os.path.join(base_path, f'{subject_id}_seg.nii')    # Changed to .nii
            }
            parsed_paths.append(paths)
        return parsed_paths

    def __len__(self) -> int:
        return len(self.data_paths)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        data_info = self.data_paths[idx]

        # Load all modalities and segmentation mask using nibabel
        # These will now correctly look for .nii files
        flair = nib.load(data_info['flair']).get_fdata()
        t1 = nib.load(data_info['t1']).get_fdata()
        t1ce = nib.load(data_info['t1ce']).get_fdata()
        t2 = nib.load(data_info['t2']).get_fdata()
        seg = nib.load(data_info['seg']).get_fdata()

        # Stack modalities (C, D, H, W)
        combined_image = np.stack([flair, t1, t1ce, t2], axis=0) # Shape (4, D, H, W)

        # --- Normalization (Z-score standardization per modality) ---
        normalized_images = []
        for i in range(combined_image.shape[0]): # Iterate through channels/modalities
            channel_data = combined_image[i]
            mean_val = np.mean(channel_data)
            std_val = np.std(channel_data)
            if std_val > 1e-8: # Avoid division by zero
                normalized_images.append((channel_data - mean_val) / std_val)
            else:
                normalized_images.append(channel_data) # If all values are same, no change
        combined_image_normalized = np.stack(normalized_images, axis=0).astype(np.float32)

        # --- Resizing/Cropping to patch_size (128x128x128) ---
        # This performs a central crop if image is larger, or pads if smaller.
        
        # Apply cropping/padding to each channel and the mask
        image_patch = np.zeros((combined_image_normalized.shape[0], *self.patch_size), dtype=combined_image_normalized.dtype)
        for c in range(combined_image_normalized.shape[0]):
            image_patch[c] = self._crop_or_pad_3d(combined_image_normalized[c], self.patch_size)
        
        mask_patch = self._crop_or_pad_3d(seg, self.patch_size)

        # --- Data Augmentation (only for training) ---
        if self.train:
            # Random 3D rotation
            if random.random() < 0.5:
                angle = random.uniform(-10, 10) # degrees
                # Rotate around a random pair of axes (for 3D volume, axes are 0, 1, 2)
                # Ensure axes are chosen correctly for a (D,H,W) volume
                axes_choice = random.choice([(0,1), (0,2), (1,2)]) # (D,H), (D,W), (H,W)
                
                # Apply rotation to each channel of image and to segmentation
                for c in range(image_patch.shape[0]):
                    image_patch[c] = rotate(image_patch[c], angle, axes=axes_choice, reshape=False, mode='nearest', order=1)
                mask_patch = rotate(mask_patch, angle, axes=axes_choice, reshape=False, mode='nearest', order=0) # order=0 for nearest neighbor for labels

            # Random flips
            if random.random() < 0.5: # Flip along depth axis (axis 0 for (D,H,W))
                image_patch = np.flip(image_patch, axis=1).copy() # axis 1 for (C,D,H,W)
                mask_patch = np.flip(mask_patch, axis=0).copy()
            if random.random() < 0.5: # Flip along height axis (axis 1 for (D,H,W))
                image_patch = np.flip(image_patch, axis=2).copy() # axis 2 for (C,D,H,W)
                mask_patch = np.flip(mask_patch, axis=1).copy()
            if random.random() < 0.5: # Flip along width axis (axis 2 for (D,H,W))
                image_patch = np.flip(image_patch, axis=3).copy() # axis 3 for (C,D,H,W)
                mask_patch = np.flip(mask_patch, axis=2).copy()


        # --- Post-processing for segmentation mask ---
        # Remap BraTS label 4 to 3 for contiguous classes (0, 1, 2, 3)
        mask_patch[mask_patch == 4] = 3
        mask_patch = mask_patch.astype(np.int64) # Ensure it's long for CrossEntropyLoss

        # Convert to PyTorch tensors
        image_tensor = torch.from_numpy(image_patch).float()
        mask_tensor = torch.from_numpy(mask_patch).long()

        return image_tensor, mask_tensor

    def _crop_or_pad_3d(self, volume: np.ndarray, target_size: Tuple[int, int, int]) -> np.ndarray:
        """
        Helper method to crop or pad a single 3D volume to a target size.
        """
        current_D, current_H, current_W = volume.shape
        target_D, target_H, target_W = target_size

        output_volume = np.zeros(target_size, dtype=volume.dtype)

        # Calculate cropping/padding for each dimension
        d_start_out = max(0, (target_D - current_D) // 2)
        h_start_out = max(0, (target_H - current_H) // 2)
        w_start_out = max(0, (target_W - current_W) // 2)

        d_start_in = max(0, (current_D - target_D) // 2)
        h_start_in = max(0, (current_H - target_H) // 2)
        w_start_in = max(0, (current_W - target_W) // 2)

        d_end_in = d_start_in + min(current_D, target_D)
        h_end_in = h_start_in + min(current_H, target_H)
        w_end_in = w_start_in + min(current_W, target_W)

        d_end_out = d_start_out + (d_end_in - d_start_in)
        h_end_out = h_start_out + (h_end_in - h_start_in)
        w_end_out = w_start_out + (w_end_in - w_start_in)

        output_volume[d_start_out:d_end_out, h_start_out:h_end_out, w_start_out:w_end_out] = \
            volume[d_start_in:d_end_in, h_start_in:h_end_in, w_start_in:w_end_in]
        
        return output_volume

print("Cell 3 executed: BraTS Dataset Class")

Cell 3 executed: BraTS Dataset Class


In [5]:
# Cell 4: SLCA-UNet Model Architecture

class ConvBlock(nn.Module):
    """
    A convolution block composed of 3D Conv, BatchNorm, and LeakyReLU.
    """
    def __init__(self, in_features: int, out_features: int):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv3d(in_features, out_features, kernel_size=3, padding=1, bias=False)
        self.norm = nn.BatchNorm3d(out_features)
        self.relu = nn.LeakyReLU(negative_slope=0.01, inplace=True)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.relu(self.norm(self.conv(x)))

class BottleneckBlock(nn.Module):
    """
    A bottleneck block with two convolutional layers and a dropout layer.
    """
    def __init__(self, in_features: int, out_features: int, dropout_rate: float):
        super(BottleneckBlock, self).__init__()
        self.conv1 = ConvBlock(in_features, out_features)
        self.conv2 = ConvBlock(out_features, out_features)
        self.dropout = nn.Dropout3d(dropout_rate)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.conv2(x)
        return self.dropout(x)

class SCALayer(nn.Module):
    """
    Spatial and Channel Attention Layer.
    Combines global spatial information with channel-wise dependencies.
    """
    def __init__(self, in_features: int):
        super(SCALayer, self).__init__()
        # Spatial Attention: Global Average Pooling -> Conv -> Sigmoid
        self.spatial_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d(1), # Global Avg Pool (D, H, W to 1, 1, 1)
            nn.Conv3d(in_features, in_features // 8, kernel_size=1, bias=False), # Reduced channels
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            nn.Conv3d(in_features // 8, in_features, kernel_size=1, bias=False), # Restore channels
            nn.Sigmoid()
        )
        # Channel Attention: Squeeze and Excitation (AvgPool -> FC -> ReLU -> FC -> Sigmoid)
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool3d(1), # Squeeze operation
            nn.Conv3d(in_features, in_features // 8, kernel_size=1, bias=False),
            nn.LeakyReLU(negative_slope=0.01, inplace=True),
            nn.Conv3d(in_features // 8, in_features, kernel_size=1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        spatial_att = self.spatial_attention(x)
        channel_att = self.channel_attention(x)
        # Apply attention to input feature map
        return x * spatial_att * channel_att # Element-wise multiplication

class SLCAUNet(nn.Module):
    """
    Scleral Residue Class Attention UNet for 3D medical image segmentation.
    Inspired by UNet with added attention mechanisms (SCALayer).
    """
    def __init__(self, in_channels: int, num_classes: int, init_features: int = 32, dropout_rate: float = 0.3):
        super(SLCAUNet, self).__init__()
        self.num_classes = num_classes

        # Encoder path
        self.encoder1 = BottleneckBlock(in_channels, init_features, dropout_rate)
        self.pool1 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.sca1 = SCALayer(init_features) # Attention after first block

        self.encoder2 = BottleneckBlock(init_features, init_features * 2, dropout_rate)
        self.pool2 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.sca2 = SCALayer(init_features * 2) # Attention after second block

        self.encoder3 = BottleneckBlock(init_features * 2, init_features * 4, dropout_rate)
        self.pool3 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.sca3 = SCALayer(init_features * 4) # Attention after third block

        self.encoder4 = BottleneckBlock(init_features * 4, init_features * 8, dropout_rate)
        self.pool4 = nn.MaxPool3d(kernel_size=2, stride=2)
        self.sca4 = SCALayer(init_features * 8) # Attention after fourth block

        # Bottleneck (latent space)
        self.bottleneck = BottleneckBlock(init_features * 8, init_features * 16, dropout_rate)
        self.sca_bottleneck = SCALayer(init_features * 16) # Attention in bottleneck

        # Decoder path
        self.upconv4 = nn.ConvTranspose3d(init_features * 16, init_features * 8, kernel_size=2, stride=2)
        self.decoder4 = BottleneckBlock(init_features * 16, init_features * 8, dropout_rate) # (init_features * 8 (from upconv) + init_features * 8 (from skip))

        self.upconv3 = nn.ConvTranspose3d(init_features * 8, init_features * 4, kernel_size=2, stride=2)
        self.decoder3 = BottleneckBlock(init_features * 8, init_features * 4, dropout_rate)

        self.upconv2 = nn.ConvTranspose3d(init_features * 4, init_features * 2, kernel_size=2, stride=2)
        self.decoder2 = BottleneckBlock(init_features * 4, init_features * 2, dropout_rate)

        self.upconv1 = nn.ConvTranspose3d(init_features * 2, init_features, kernel_size=2, stride=2)
        self.decoder1 = BottleneckBlock(init_features * 2, init_features, dropout_rate)

        # Output layer
        self.out_conv = nn.Conv3d(init_features, num_classes, kernel_size=1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Encoder
        enc1 = self.encoder1(x)
        enc1_att = self.sca1(enc1) # Apply attention
        pool1 = self.pool1(enc1_att)

        enc2 = self.encoder2(pool1)
        enc2_att = self.sca2(enc2)
        pool2 = self.pool2(enc2_att)

        enc3 = self.encoder3(pool2)
        enc3_att = self.sca3(enc3)
        pool3 = self.pool3(enc3_att)

        enc4 = self.encoder4(pool3)
        enc4_att = self.sca4(enc4)
        pool4 = self.pool4(enc4_att)

        # Bottleneck
        bottleneck = self.bottleneck(pool4)
        bottleneck_att = self.sca_bottleneck(bottleneck)

        # Decoder
        up4 = self.upconv4(bottleneck_att)
        # print(f"Shape enc4_att: {enc4_att.shape}, up4: {up4.shape}")
        # Ensure dimensions match before concatenation if upsampling caused slight mismatch
        # This can happen if input dimensions are not perfectly divisible by 2 at each step.
        # For our fixed PATCH_SIZE (128,128,128), this should align.
        dec4 = self.decoder4(torch.cat([enc4_att, up4], dim=1))

        up3 = self.upconv3(dec4)
        dec3 = self.decoder3(torch.cat([enc3_att, up3], dim=1))

        up2 = self.upconv2(dec3)
        dec2 = self.decoder2(torch.cat([enc2_att, up2], dim=1))

        up1 = self.upconv1(dec2)
        dec1 = self.decoder1(torch.cat([enc1_att, up1], dim=1))

        return self.out_conv(dec1)

    def summary(self):
        """Prints a summary of the model architecture."""
        try:
            # We need a dummy input to trace the model and get summary
            dummy_input = torch.randn(1, self.encoder1.conv1.conv.in_channels, 128, 128, 128).to(device)
            # Use torchinfo.summary if available, otherwise print basic info
            if 'torchinfo' in sys.modules: # Check if torchinfo is imported
                from torchinfo import summary
                summary(self, input_size=dummy_input.shape)
            else:
                print(self)
                print(f"Model on device: {next(self.parameters()).device}")
                print(f"Total parameters: {sum(p.numel() for p in self.parameters() if p.requires_grad)}")
        except Exception as e:
            print(f"Could not generate detailed model summary: {e}")
            print(self) # Fallback to printing the model structure


# For model.summary() to work with torchinfo, you might need to install it.
# !pip install torchinfo # Uncomment and run if you want a detailed summary
import sys
if 'torchinfo' not in sys.modules:
    try:
        pass # Not installing here, assuming it's an optional detailed summary
    except ImportError:
        logger.warning("torchinfo not found. Model summary will be basic.")


print("Cell 4 executed: SLCA-UNet Model Architecture")

Cell 4 executed: SLCA-UNet Model Architecture


In [6]:
# Cell 5: Loss Functions

# Assuming this is your DiceLoss class definition
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, num_classes: int, smooth: float = 1e-6, class_weights: Optional[torch.Tensor] = None):
        super().__init__()
        self.num_classes = num_classes
        self.smooth = smooth
        # CORRECTED LINE: Directly assign class_weights, as it's already a tensor or None
        self.class_weights = class_weights

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        # (Rest of your DiceLoss forward method remains the same)
        # Simplified Dice Loss for demonstration
        probs = F.softmax(inputs, dim=1)
        targets_one_hot = F.one_hot(targets, num_classes=self.num_classes).permute(0, 4, 1, 2, 3).float()

        intersection = (probs * targets_one_hot).sum(dim=[2, 3, 4])
        union = (probs + targets_one_hot).sum(dim=[2, 3, 4])

        dice = (2. * intersection + self.smooth) / (union + self.smooth)

        if self.class_weights is not None:
            # Ensure class_weights broadcasting matches dice tensor
            # If dice is (N, C), class_weights needs to be (C,) or (1, C)
            dice = dice * self.class_weights.unsqueeze(0) # Apply weight per class

        return (1.0 - dice).mean()

# Similarly, if you have a FocalLoss that takes class_weights:
class FocalLoss(nn.Module):
    def __init__(self, num_classes: int, alpha: Optional[torch.Tensor] = None, gamma: float = 2.0, reduction: str = 'mean'):
        super().__init__()
        self.num_classes = num_classes
        # Corrected line for alpha/class_weights if FocalLoss uses it this way
        self.alpha = alpha # This should now directly be the tensor or None
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        
        # Ensure alpha applies correctly, often based on target class
        if self.alpha is not None:
            focal_loss = self.alpha[targets] * ((1 - pt) ** self.gamma) * ce_loss
        else:
            focal_loss = ((1 - pt) ** self.gamma) * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss
class FocalLoss(nn.Module):
    """
    Focal Loss for highly imbalanced datasets.
    Reduces the weight of well-classified examples.
    """
    def __init__(self, alpha: float = 0.25, gamma: float = 2.0, num_classes: int = 4, class_weights: Optional[List[float]] = None):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.num_classes = num_classes
        # Convert class weights to a tensor
        self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device) if class_weights else None

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        # Inputs (N, C, D, H, W) -> logits
        # Targets (N, D, H, W) -> class labels (long type)

        # Ensure inputs are in float and targets are long
        inputs = inputs.float()
        targets = targets.long()

        # Calculate CE loss
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        # Get probabilities from logits
        pt = torch.exp(-ce_loss)

        # Calculate focal loss
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        # Apply class weights if provided
        if self.class_weights is not None:
            # Expand class_weights to match target shape
            # targets_expanded = targets.unsqueeze(1).float()
            # weighted_focal_loss = focal_loss * self.class_weights[targets].float() # This might be wrong for 3D
            # Need to apply weight based on the class of each pixel in the target
            weights_map = self.class_weights[targets.long()]
            focal_loss = focal_loss * weights_map
        
        return focal_loss.mean()

class CombinedLoss(nn.Module):
    """
    Combines Cross-Entropy Loss, Dice Loss, and optionally Focal Loss.
    """
    def __init__(self, ce_weight: float = 0.5, dice_weight: float = 0.5, focal_weight: float = 0.0,
                 num_classes: int = 4, class_weights: Optional[List[float]] = None):
        super(CombinedLoss, self).__init__()
        self.ce_weight = ce_weight
        self.dice_weight = dice_weight
        self.focal_weight = focal_weight
        self.num_classes = num_classes
        self.class_weights_tensor = class_weights # class_weights is already a tensor from Cell 10
        self.cross_entropy_loss = nn.CrossEntropyLoss(weight=self.class_weights_tensor)
        self.dice_loss = DiceLoss(num_classes=num_classes, class_weights=class_weights) # Pass class_weights
        if focal_weight > 0:
            self.focal_loss = FocalLoss(num_classes=num_classes, class_weights=class_weights) # Pass class_weights
        else:
            self.focal_loss = None

    def forward(self, inputs: torch.Tensor, targets: torch.Tensor) -> torch.Tensor:
        total_loss = 0.0

        if self.ce_weight > 0:
            # CrossEntropyLoss expects inputs (N, C, D, H, W) and targets (N, D, H, W)
            ce = self.cross_entropy_loss(inputs, targets)
            total_loss += self.ce_weight * ce

        if self.dice_weight > 0:
            dice = self.dice_loss(inputs, targets)
            total_loss += self.dice_weight * dice

        if self.focal_loss and self.focal_weight > 0:
            focal = self.focal_loss(inputs, targets)
            total_loss += self.focal_weight * focal

        return total_loss

print("Cell 5 executed: Loss Functions")

Cell 5 executed: Loss Functions


In [7]:
# Cell 6: SLCA-UNet Trainer (Updated to collect Sensitivity and Specificity)

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
from pathlib import Path
from typing import Any, Dict, List, Tuple, Optional

# Assuming 'logger' and 'SegmentationMetrics' are defined elsewhere (e.g., in Cell 1 or 5)
# For this code to run, ensure you have:
# - from your_project.metrics import SegmentationMetrics # Or wherever it's located
# - from your_project.utils import logger # Or your logging setup

# Placeholder for logger and SegmentationMetrics if they are not globally available
# In a real notebook, these would come from earlier cells.
try:
    _ = logger # Check if logger is defined
except NameError:
    import logging
    logger = logging.getLogger(__name__)
    if not logger.handlers:
        logging.basicConfig(level=logging.INFO)
try:
    _ = SegmentationMetrics # Check if SegmentationMetrics is defined
except NameError:
    # This is a dummy class. You MUST replace this with your actual SegmentationMetrics class
    # that has an 'evaluate_dataset' method returning 'Dice Score', 'Hausdorff Distance 95',
    # 'Sensitivity', and 'Specificity' keys in its output dictionary.
    class SegmentationMetrics:
        def __init__(self, num_classes: int):
            self.num_classes = num_classes
        def evaluate_dataset(self, all_preds: List[np.ndarray], all_targets: List[np.ndarray]) -> Dict[str, Dict[str, float]]:
            # Dummy implementation: In a real scenario, this calculates actual metrics
            # For demonstration, return dummy values.
            return {
                'Dice Score': {'mean_overall': np.random.rand() * 0.2 + 0.7}, # e.g., 0.7 to 0.9
                'Hausdorff Distance 95': {'mean_overall': np.random.rand() * 5.0 + 5.0}, # e.g., 5.0 to 10.0
                'Sensitivity': {'mean_overall': np.random.rand() * 0.2 + 0.75}, # e.g., 0.75 to 0.95
                'Specificity': {'mean_overall': np.random.rand() * 0.05 + 0.94}, # e.g., 0.94 to 0.99
            }
    import numpy as np # Needed for the dummy SegmentationMetrics

class SLCAUNetTrainer:
    """
    Manages the training process for the SLCA-UNet model.
    Includes training loop, validation, checkpointing, and logging.
    """
    def __init__(self, model: nn.Module, train_loader: DataLoader, val_loader: DataLoader,
                 criterion: nn.Module, optimizer: optim.Optimizer, scheduler: Any,
                 device: torch.device, num_classes: int, gradient_clip_value: float = 1.0,
                 mixed_precision: bool = True, log_dir: str = './logs', checkpoint_dir: str = './checkpoints'):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.num_classes = num_classes
        self.gradient_clip_value = gradient_clip_value
        self.mixed_precision = mixed_precision
        self.scaler = torch.cuda.amp.GradScaler() if mixed_precision and self.device.type == 'cuda' else None

        self.log_dir = Path(log_dir)
        self.checkpoint_dir = Path(checkpoint_dir)
        self.log_dir.mkdir(parents=True, exist_ok=True)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)

        self.best_val_dice = -1.0
        self.best_val_hd95 = float('inf') # Hausdorff Distance: lower is better

        # Initialize history as None in __init__. It will be fully set up in the train() method.
        self.history = None

        logger.info(f"Trainer initialized. Mixed Precision: {self.mixed_precision}")

    def _train_one_epoch(self) -> float:
        """Trains the model for one epoch."""
        self.model.train()
        running_loss = 0.0
        pbar = tqdm(self.train_loader, desc="Training", leave=False)
        for i, (images, masks) in enumerate(pbar):
            images, masks = images.to(self.device), masks.to(self.device)

            self.optimizer.zero_grad()

            with torch.cuda.amp.autocast(enabled=self.mixed_precision):
                outputs = self.model(images)
                loss = self.criterion(outputs, masks)

            if self.scaler:
                self.scaler.scale(loss).backward()
                self.scaler.unscale_(self.optimizer) # Unscale gradients before clipping
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.gradient_clip_value)
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.gradient_clip_value)
                self.optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        avg_loss = running_loss / len(self.train_loader)
        return avg_loss

    # MODIFIED: _validate_one_epoch now returns Sensitivity and Specificity
    def _validate_one_epoch(self) -> Tuple[float, float, float, float, float]: # Added two more float types
        """Validates the model for one epoch."""
        self.model.eval()
        running_val_loss = 0.0
        all_preds = []
        all_targets = []

        with torch.no_grad():
            pbar = tqdm(self.val_loader, desc="Validation", leave=False)
            for images, masks in pbar:
                images, masks = images.to(self.device), masks.to(self.device)

                with torch.cuda.amp.autocast(enabled=self.mixed_precision):
                    outputs = self.model(images)
                    loss = self.criterion(outputs, masks)

                running_val_loss += loss.item()

                preds = torch.argmax(F.softmax(outputs, dim=1), dim=1).cpu().numpy()
                targets = masks.cpu().numpy()

                all_preds.extend(list(preds))
                all_targets.extend(list(targets))

        avg_val_loss = running_val_loss / len(self.val_loader)

        metrics_calculator = SegmentationMetrics(num_classes=self.num_classes)
        epoch_metrics = metrics_calculator.evaluate_dataset(all_preds, all_targets)

        val_dice = epoch_metrics['Dice Score']['mean_overall']
        val_hd95 = epoch_metrics['Hausdorff Distance 95']['mean_overall']
        # ADDED: Extract Sensitivity and Specificity
        val_sensitivity = epoch_metrics['Sensitivity']['mean_overall']
        val_specificity = epoch_metrics['Specificity']['mean_overall']
        
        # MODIFIED: Return Sensitivity and Specificity
        return avg_val_loss, val_dice, val_hd95, val_sensitivity, val_specificity

    def train(self, num_epochs: int, save_interval: int = 5) -> Dict[str, List[float]]:
        """
        Runs the full training loop.
        Args:
            num_epochs: Total number of epochs to train for.
            save_interval: Save a model checkpoint every `save_interval` epochs.
        Returns:
            A dictionary containing lists of training/validation losses and metrics per epoch.
        """
        # CORRECTED: Initialize self.history here as an instance attribute
        self.history = {
            'train_losses': [],
            'val_losses': [],
            'val_dice_scores': [],
            'val_hd95_scores': [],
            'learning_rates': [],
            'val_sensitivity_scores': [], # ADDED: For storing sensitivity
            'val_specificity_scores': []  # ADDED: For storing specificity
        }

        logger.info(f"Starting training for {num_epochs} epochs.")
        for epoch in range(1, num_epochs + 1):
            start_time = time.time()
            logger.info(f"Epoch {epoch}/{num_epochs}")

            # Training phase
            train_loss = self._train_one_epoch()
            self.history['train_losses'].append(train_loss)

            # Validation phase
            # MODIFIED: Capture Sensitivity and Specificity
            val_loss, val_dice, val_hd95, val_sensitivity, val_specificity = self._validate_one_epoch()
            self.history['val_losses'].append(val_loss)
            self.history['val_dice_scores'].append(val_dice)
            self.history['val_hd95_scores'].append(val_hd95)
            self.history['val_sensitivity_scores'].append(val_sensitivity) # ADDED: Append sensitivity
            self.history['val_specificity_scores'].append(val_specificity) # ADDED: Append specificity

            # Record learning rate
            current_lr = self.optimizer.param_groups[0]['lr']
            self.history['learning_rates'].append(current_lr)

            end_time = time.time()
            epoch_duration = end_time - start_time

            # MODIFIED: Include Sensitivity and Specificity in logger info
            logger.info(
                f"Epoch {epoch} finished in {epoch_duration:.2f}s | "
                f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                f"Val Dice: {val_dice:.4f} | Val HD95: {val_hd95:.2f} | "
                f"Val Sens: {val_sensitivity:.4f} | Val Spec: {val_specificity:.4f} | " # Added
                f"LR: {current_lr:.6f}"
            )

            # Learning rate scheduler step
            if self.scheduler:
                if isinstance(self.scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                    self.scheduler.step(val_dice)
                else:
                    self.scheduler.step()

            # Checkpoint saving
            if val_dice > self.best_val_dice:
                self.best_val_dice = val_dice
                self.best_val_hd95 = val_hd95
                self._save_checkpoint(f'best_model.pth', is_best=True)
                logger.info(f"New best model saved with Dice: {val_dice:.4f}, HD95: {val_hd95:.2f}")

            if epoch % save_interval == 0:
                self._save_checkpoint(f'epoch_{epoch:03d}.pth')
                logger.info(f"Checkpoint saved for epoch {epoch}")

        logger.info("Training complete.")
        return self.history

    def _save_checkpoint(self, filename: str, is_best: bool = False) -> None:
        """Saves the model's current state."""
        state = {
            'epoch': len(self.history['train_losses']),
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'best_val_dice': self.best_val_dice,
            'best_val_hd95': self.best_val_hd95,
            'scaler_state_dict': self.scaler.state_dict() if self.scaler else None,
            # It's good practice to save history to checkpoints for resuming training
            'history': self.history # ADDED: Save training history
        }
        filepath = self.checkpoint_dir / filename
        torch.save(state, filepath)
        if is_best:
            torch.save(state, self.checkpoint_dir / 'best_model.pth')

    def load_checkpoint(self, filepath: str) -> None:
        """Loads a model checkpoint."""
        logger.info(f"Loading checkpoint from {filepath}")
        checkpoint = torch.load(filepath, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        if self.scaler and checkpoint['scaler_state_dict']:
            self.scaler.load_state_dict(checkpoint['scaler_state_dict'])
        self.best_val_dice = checkpoint.get('best_val_dice', -1.0)
        self.best_val_hd95 = checkpoint.get('best_val_hd95', float('inf'))
        # MODIFIED: Load history from checkpoint if available
        self.history = checkpoint.get('history', {
            'train_losses': [], 'val_losses': [], 'val_dice_scores': [],
            'val_hd95_scores': [], 'learning_rates': [],
            'val_sensitivity_scores': [], 'val_specificity_scores': []
        })
        logger.info(f"Checkpoint loaded. Best Val Dice: {self.best_val_dice:.4f}, Best Val HD95: {self.best_val_hd95:.2f}")

print("Cell 6 executed: SLCA-UNet Trainer (Updated)")

Cell 6 executed: SLCA-UNet Trainer (Updated)


In [8]:
# Cell 7: SLCA-UNet Inference

class SLCAUNetInference:
    """
    Handles inference for the SLCA-UNet model, including loading the model,
    applying sliding window inference, and Test-Time Augmentation (TTA).
    """
    def __init__(self, model_path: str, model_config: Dict[str, Any], device: torch.device):
        self.device = device
        # Pass relevant config to SLCAUNet, not the whole model_config if it contains extra keys
        # Assuming model_config has 'in_channels' and 'num_classes'
        self.model = SLCAUNet(
            in_channels=model_config.get('in_channels', 4), # Default to 4 if not specified
            num_classes=model_config.get('num_classes', 4)  # Default to 4 if not specified
            # Add other SLCAUNet __init__ args if needed, like init_features, dropout_rate
        ).to(self.device)

        # Load the model state dictionary from the checkpoint file
        checkpoint = torch.load(model_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval() # Set model to evaluation mode

        # Ensure patch_size is consistently retrieved or set
        # This patch_size should match what the dataset prepares
        self.patch_size = model_config.get('patch_size', (128, 128, 128))
        logger.info(f"Inference engine initialized. Model loaded from {model_path} with patch size {self.patch_size}.")

    def _predict_patch(self, image_patch: torch.Tensor, use_tta: bool) -> Tuple[np.ndarray, np.ndarray]:
        """
        Performs prediction on a single image patch (C, D, H, W).
        Optionally applies Test-Time Augmentation (TTA).
        Returns predicted class labels (D, H, W) numpy array and (C, D, H, W) probabilities.
        """
        image_patch = image_patch.to(self.device) # Move to device

        with torch.no_grad():
            if use_tta:
                # Original prediction
                logits = self.model(image_patch.unsqueeze(0)).squeeze(0) # Add batch dim, remove after
                predictions_sum = F.softmax(logits, dim=0).cpu().numpy() # (C, D, H, W) probabilities

                # TTA: flips (spatial dimensions are 1, 2, 3 for (C, D, H, W))
                # 0: Channel, 1: Depth, 2: Height, 3: Width
                spatial_flip_dims = [
                    (1,), (2,), (3,),        # Single axis flips (D, H, W)
                    (1,2), (1,3), (2,3),     # Double axis flips (D,H), (D,W), (H,W)
                    (1,2,3)                  # Triple axis flip (D,H,W)
                ]
                
                num_augmentations = 1 # For original prediction
                for dim_flips in spatial_flip_dims:
                    flipped_image = image_patch
                    for dim in dim_flips:
                        flipped_image = torch.flip(flipped_image, dims=[dim])
                    
                    flipped_logits = self.model(flipped_image.unsqueeze(0)).squeeze(0)
                    flipped_preds = F.softmax(flipped_logits, dim=0).cpu().numpy()

                    # Flip predictions back to original orientation
                    for dim in dim_flips:
                        # numpy.flip uses 0-indexed axes directly.
                        # If torch.flip used dim 1 (Depth), numpy.flip uses axis 1.
                        flipped_preds = np.flip(flipped_preds, axis=dim)
                    
                    predictions_sum += flipped_preds
                    num_augmentations += 1
                
                averaged_predictions = predictions_sum / num_augmentations

            else:
                logits = self.model(image_patch.unsqueeze(0)).squeeze(0)
                averaged_predictions = F.softmax(logits, dim=0).cpu().numpy()
        
        # Get the class with the highest probability
        predicted_labels = np.argmax(averaged_predictions, axis=0) # (D, H, W)
        return predicted_labels, averaged_predictions # Return labels and probabilities

    def batch_inference(self, dataset: Dataset, output_dir: str,
                        use_sliding_window: bool = True, use_tta: bool = False,
                        window_overlap: float = 0.5) -> Dict[str, Dict[str, np.ndarray]]:
        """
        Performs inference on a full dataset.
        If use_sliding_window is True, applies sliding window inference.
        Returns a dictionary of results for each subject.
        """
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        results = {}

        pbar = tqdm(dataset, desc="Batch Inference", leave=True)
        for i, (images_tensor, ground_truth_mask_tensor) in enumerate(pbar):
            subject_id = dataset.data_paths[i]['id'] # Get subject ID from dataset

            full_image_volume = images_tensor # This is already preprocessed (cropped/padded to patch_size)

            predicted_prob_map = None # Initialize to ensure it's always set

            if use_sliding_window:
                predicted_prob_map = self._sliding_window_inference(full_image_volume, use_tta, window_overlap)
                predicted_labels = np.argmax(predicted_prob_map, axis=0)
            else:
                # If not sliding window, the dataset should be providing patches of self.patch_size
                predicted_labels, predicted_prob_map = self._predict_patch(full_image_volume, use_tta)

            # Save predicted segmentation mask
            save_path_labels = os.path.join(output_dir, f'{subject_id}_pred_labels.nii.gz')
            # Assuming MedicalImageUtils.save_nifti exists and works as expected
            MedicalImageUtils.save_nifti(predicted_labels.astype(np.uint8), save_path_labels)

            # Store results
            results[subject_id] = {
                'labels': predicted_labels,
                'probabilities': predicted_prob_map, # Now always defined
                'ground_truth_labels': ground_truth_mask_tensor.numpy()
            }
            pbar.set_postfix(subject=subject_id, status="Processed")

        return results

    def _sliding_window_inference(self, image_tensor: torch.Tensor, use_tta: bool, overlap: float = 0.5) -> np.ndarray:
        """
        Performs sliding window inference on a 3D image volume.
        image_tensor shape: (C, D, H, W)
        """
        _, D, H, W = image_tensor.shape
        # Calculate step sizes ensuring at least 1-pixel overlap if patch_size is large
        d_step = max(1, int(self.patch_size[0] * (1 - overlap)))
        h_step = max(1, int(self.patch_size[1] * (1 - overlap)))
        w_step = max(1, int(self.patch_size[2] * (1 - overlap)))

        # Initialize output probability map and count map for averaging overlaps
        # self.model.num_classes should be available from the loaded model
        num_classes_model = self.model.num_classes 
        output_prob_map = np.zeros((num_classes_model, D, H, W), dtype=np.float32)
        count_map = np.zeros((D, H, W), dtype=np.float32)

        # Generate window starting coordinates
        # Ensure the last window always includes the end of the dimension
        d_starts = list(range(0, D - self.patch_size[0] + 1, d_step))
        if (D - self.patch_size[0]) % d_step != 0: d_starts.append(D - self.patch_size[0])
        d_starts = sorted(list(set(d_starts))) # Remove duplicates and sort

        h_starts = list(range(0, H - self.patch_size[1] + 1, h_step))
        if (H - self.patch_size[1]) % h_step != 0: h_starts.append(H - self.patch_size[1])
        h_starts = sorted(list(set(h_starts)))

        w_starts = list(range(0, W - self.patch_size[2] + 1, w_step))
        if (W - self.patch_size[2]) % w_step != 0: w_starts.append(W - self.patch_size[2])
        w_starts = sorted(list(set(w_starts)))

        total_patches = len(d_starts) * len(h_starts) * len(w_starts)
        logger.info(f"Sliding window inference: {total_patches} patches to process for volume of shape ({D},{H},{W}).")

        patch_idx = 0
        for d_start in d_starts:
            for h_start in h_starts:
                for w_start in w_starts:
                    d_end, h_end, w_end = d_start + self.patch_size[0], h_start + self.patch_size[1], w_start + self.patch_size[2]
                    
                    # Extract patch
                    patch_image = image_tensor[:, d_start:d_end, h_start:h_end, w_start:w_end]

                    # Predict probabilities for the patch
                    # We only need the probabilities here for averaging
                    _, patch_probs = self._predict_patch(patch_image, use_tta) # patch_probs is (C, P_D, P_H, P_W)

                    # Add to cumulative probability map and count map
                    output_prob_map[:, d_start:d_end, h_start:h_end, w_start:w_end] += patch_probs
                    count_map[d_start:d_end, h_start:h_end, w_start:w_end] += 1.0
                    
                    patch_idx += 1
                    if patch_idx % 100 == 0 or patch_idx == total_patches:
                        logger.debug(f"  Processed {patch_idx}/{total_patches} patches.")

        # Handle zero counts (should not happen with overlap but as a safeguard)
        count_map[count_map == 0] = 1.0 # Avoid division by zero

        # Average overlapping predictions
        averaged_prob_map = output_prob_map / count_map

        return averaged_prob_map

print("Cell 7 executed: SLCA-UNet Inference")

Cell 7 executed: SLCA-UNet Inference


In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, Optional, List
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class SLCAUNet:
    """
    Stacked Convolution Layered Channel Attention UNet (SLCA-UNet)
    Implementation for brain tumor segmentation.
    """
    
    def __init__(self, 
                 input_shape: Tuple[int, int, int] = (256, 256, 4),
                 num_classes: int = 4,
                 initial_filters: int = 32,
                 filter_multiplier: int = 2,
                 encoder_layers: int = 4,
                 se_reduction_ratio: int = 16,
                 dropout_rate: float = 0.5):
        """
        Initialize SLCA-UNet model.
        
        Args:
            input_shape: Input tensor shape (H, W, C)
            num_classes: Number of output classes
            initial_filters: Initial number of filters
            filter_multiplier: Filter multiplication factor per layer
            encoder_layers: Number of encoder layers
            se_reduction_ratio: Squeeze-and-excitation reduction ratio
            dropout_rate: Dropout rate
        """
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.initial_filters = initial_filters
        self.filter_multiplier = filter_multiplier
        self.encoder_layers = encoder_layers
        self.se_reduction_ratio = se_reduction_ratio
        self.dropout_rate = dropout_rate
        
        # Calculate filter sizes for each layer
        self.filters = [initial_filters * (filter_multiplier ** i) 
                        for i in range(encoder_layers)]
        
    def residual_dense_block(self, inputs: tf.Tensor, filters: int, name: str) -> tf.Tensor:
        """
        Implement residual dense block with dense connections and residual connections.
        
        Args:
            inputs: Input tensor
            filters: Number of filters
            name: Block name
            
        Returns:
            Output tensor
        """
        with tf.name_scope(f"{name}_residual_dense"):
            # First convolution branch
            c1 = layers.Conv2D(filters, 3, padding='same', 
                               kernel_initializer='he_normal',
                               name=f"{name}_conv1")(inputs)
            c1 = layers.BatchNormalization(name=f"{name}_bn1")(c1)
            c1 = layers.ReLU(name=f"{name}_relu1")(c1)
            
            # Second convolution branch
            c2 = layers.Conv2D(filters, 3, padding='same',
                               kernel_initializer='he_normal',
                               name=f"{name}_conv2")(c1)
            c2 = layers.BatchNormalization(name=f"{name}_bn2")(c2)
            c2 = layers.ReLU(name=f"{name}_relu2")(c2)
            
            # Third convolution branch
            c3 = layers.Conv2D(filters, 3, padding='same',
                               kernel_initializer='he_normal',
                               name=f"{name}_conv3")(c2)
            c3 = layers.BatchNormalization(name=f"{name}_bn3")(c3)
            c3 = layers.ReLU(name=f"{name}_relu3")(c3)
            
            # Residual connection (element-wise addition)
            # Ensure input channels match filters for residual addition
            if inputs.shape[-1] != filters:
                inputs = layers.Conv2D(filters, 1, padding='same',
                                       kernel_initializer='he_normal',
                                       name=f"{name}_skip_conv")(inputs)
                inputs = layers.BatchNormalization(name=f"{name}_skip_bn")(inputs)
            
            # Element-wise addition of c2 and c3, then add residual
            output = layers.Add(name=f"{name}_add1")([c2, c3])
            output = layers.Add(name=f"{name}_residual_add")([output, inputs])
            
            return output
        
    def stacked_convolution_module(self, inputs: tf.Tensor, filters: int, name: str) -> tf.Tensor:
        """
        Implement stacked convolution module with different dilation rates.
        
        Args:
            inputs: Input tensor
            filters: Number of filters
            name: Module name
            
        Returns:
            Output tensor
        """
        with tf.name_scope(f"{name}_stacked_conv"):
            # Atrous convolutions with different dilation rates
            conv_6 = layers.Conv2D(filters, 3, padding='same', dilation_rate=6,
                                   kernel_initializer='he_normal',
                                   name=f"{name}_conv_6")(inputs)
            conv_6 = layers.BatchNormalization(name=f"{name}_bn_6")(conv_6)
            conv_6 = layers.ReLU(name=f"{name}_relu_6")(conv_6)
            
            conv_12 = layers.Conv2D(filters, 3, padding='same', dilation_rate=12,
                                    kernel_initializer='he_normal',
                                    name=f"{name}_conv_12")(inputs)
            conv_12 = layers.BatchNormalization(name=f"{name}_bn_12")(conv_12)
            conv_12 = layers.ReLU(name=f"{name}_relu_12")(conv_12)
            
            conv_18 = layers.Conv2D(filters, 3, padding='same', dilation_rate=18,
                                    kernel_initializer='he_normal',
                                    name=f"{name}_conv_18")(inputs)
            conv_18 = layers.BatchNormalization(name=f"{name}_bn_18")(conv_18)
            conv_18 = layers.ReLU(name=f"{name}_relu_18")(conv_18)
            
            # 1x1 convolution for upsampling
            conv_1x1 = layers.Conv2D(filters, 1, padding='same',
                                     kernel_initializer='he_normal',
                                     name=f"{name}_conv_1x1")(inputs)
            conv_1x1 = layers.BatchNormalization(name=f"{name}_bn_1x1")(conv_1x1)
            conv_1x1 = layers.ReLU(name=f"{name}_relu_1x1")(conv_1x1)
            
            # Concatenate all outputs
            concat = layers.Concatenate(name=f"{name}_concat")([conv_6, conv_12, conv_18, conv_1x1])
            
            # Channel reduction with 1x1 convolution
            output = layers.Conv2D(filters, 1, padding='same',
                                   kernel_initializer='he_normal',
                                   name=f"{name}_reduce")(concat)
            output = layers.BatchNormalization(name=f"{name}_reduce_bn")(output)
            output = layers.ReLU(name=f"{name}_reduce_relu")(output)
            
            return output
        
    def layered_attention(self, inputs: tf.Tensor, name: str) -> tf.Tensor:
        """
        Implement layered attention mechanism with multi-scale features.
        
        Args:
            inputs: Input tensor
            name: Attention name
            
        Returns:
            Attention-weighted output tensor
        """
        with tf.name_scope(f"{name}_layered_attention"):
            filters = inputs.shape[-1]
            
            # Extract multi-scale features
            # High-level features with dilation rate 2
            m1 = layers.Conv2D(filters, 3, padding='same', dilation_rate=2,
                               kernel_initializer='he_normal',
                               name=f"{name}_m1")(inputs)
            m1 = layers.BatchNormalization(name=f"{name}_m1_bn")(m1)
            m1 = layers.ReLU(name=f"{name}_m1_relu")(m1)
            
            # Low-level features with dilation rate 4
            m2 = layers.Conv2D(filters, 3, padding='same', dilation_rate=4,
                                 kernel_initializer='he_normal',
                                 name=f"{name}_m2")(inputs)
            m2 = layers.BatchNormalization(name=f"{name}_m2_bn")(m2)
            m2 = layers.ReLU(name=f"{name}_m2_relu")(m2)
            
            # Detailed features (original resolution)
            m3 = layers.Conv2D(filters, 3, padding='same', dilation_rate=1,
                                 kernel_initializer='he_normal',
                                 name=f"{name}_m3")(inputs)
            m3 = layers.BatchNormalization(name=f"{name}_m3_bn")(m3)
            m3 = layers.ReLU(name=f"{name}_m3_relu")(m3)
            
            # Combine multi-scale features
            combined = layers.Add(name=f"{name}_combine")([m1, m2, m3])
            
            # Apply attention weights
            attention = layers.Conv2D(1, 1, padding='same', activation='sigmoid',
                                     kernel_initializer='he_normal',
                                     name=f"{name}_attention")(combined)
            
            # Apply attention to original features
            output = layers.Multiply(name=f"{name}_apply_attention")([inputs, attention])
            
            return output
        
    def channel_attention_se(self, inputs: tf.Tensor, name: str) -> tf.Tensor:
        """
        Implement Squeeze-and-Excitation (SE) channel attention block.
        
        Args:
            inputs: Input tensor
            name: SE block name
            
        Returns:
            Channel attention weighted output
        """
        with tf.name_scope(f"{name}_se_block"):
            channels = inputs.shape[-1]
            
            # Global Average Pooling
            gap = layers.GlobalAveragePooling2D(name=f"{name}_gap")(inputs)
            
            # Squeeze: reduce channels
            squeeze = layers.Dense(channels // self.se_reduction_ratio,
                                   activation='relu',
                                   kernel_initializer='he_normal',
                                   name=f"{name}_squeeze")(gap)
            
            # Excitation: restore channels with sigmoid
            excitation = layers.Dense(channels,
                                      activation='sigmoid',
                                      kernel_initializer='he_normal',
                                      name=f"{name}_excitation")(squeeze)
            
            # Reshape for broadcasting
            excitation = layers.Reshape((1, 1, channels), name=f"{name}_reshape")(excitation)
            
            # Apply channel attention
            output = layers.Multiply(name=f"{name}_channel_attention")([inputs, excitation])
            
            return output
        
    def pac_module(self, inputs: tf.Tensor, name: str) -> tf.Tensor:
        """
        Implement Pixel Attention Connection (PAC) module.
        
        Args:
            inputs: Input tensor
            name: PAC module name
            
        Returns:
            PAC processed output
        """
        with tf.name_scope(f"{name}_pac_module"):
            # Apply layered attention
            layered_out = self.layered_attention(inputs, f"{name}_layered")
            
            # Apply channel attention
            channel_out = self.channel_attention_se(layered_out, f"{name}_channel")
            
            # Combine with residual connection
            output = layers.Add(name=f"{name}_pac_residual")([inputs, channel_out])
            
            return output
        
    def build_model(self) -> models.Model:
        """
        Build complete SLCA-UNet model.
        
        Returns:
            Compiled Keras model
        """
        inputs = layers.Input(shape=self.input_shape, name='input')
        
        # Encoder path
        encoder_outputs = []
        x = inputs
        
        for i in range(self.encoder_layers):
            # Residual dense block
            x = self.residual_dense_block(x, self.filters[i], f"encoder_{i}")
            encoder_outputs.append(x) # Store output before pooling for skip connection
            
            # Apply channel attention
            x = self.channel_attention_se(x, f"encoder_se_{i}")
            
            # Downsample (except for last layer)
            if i < self.encoder_layers - 1:
                x = layers.MaxPooling2D(2, name=f"encoder_pool_{i}")(x)
        
        # Bottleneck
        # x is currently at the smallest resolution (e.g., 32x32 for 256x256 input and 4 encoder layers)
        bottleneck = self.residual_dense_block(x, self.filters[-1] * 2, "bottleneck")
        bottleneck = self.pac_module(bottleneck, "bottleneck_pac")
        
        # Decoder path
        x = bottleneck # x is now the bottleneck output (e.g., 32x32, 512 channels)
        
        # Iterate from the second-deepest encoder layer's index down to 0 (i.e., skipping the last, deepest encoder output which matches bottleneck resolution)
        # For encoder_layers = 4, i will be 3, 2, 1
        for i in range(self.encoder_layers - 1, 0, -1):
            # Upsample current decoder feature map
            x = layers.UpSampling2D(2, name=f"decoder_upsample_{i}")(x)
            
            # Select the correct skip connection from encoder_outputs
            # encoder_outputs[i-1] aligns the resolutions correctly
            # Example: when i=3 (first step), we need encoder_outputs[2] (64x64)
            # when i=1 (last step), we need encoder_outputs[0] (256x256)
            current_skip_connection = encoder_outputs[i-1]
            
            # Apply stacked convolution module to the skip connection
            # Use filters[i-1] to match the corresponding encoder stage's filter count
            skip = self.stacked_convolution_module(
                current_skip_connection, self.filters[i-1], f"skip_stack_{i}")
            
            # Apply PAC module to skip connection
            skip = self.pac_module(skip, f"skip_pac_{i}")
            
            # Concatenate with the upsampled feature map
            x = layers.Concatenate(name=f"decoder_concat_{i}")([x, skip])
            
            # Apply residual dense block
            x = self.residual_dense_block(x, self.filters[i-1], f"decoder_{i}") # Adjust filters to match current stage
            
            # Apply layered attention
            x = self.layered_attention(x, f"decoder_attention_{i}")
            
            # Dropout (only during training)
            x = layers.Dropout(self.dropout_rate, name=f"decoder_dropout_{i}")(x)
        
        # Final output layer
        # x is now at the original input resolution (e.g., 256x256) with initial_filters (32)
        outputs = layers.Conv2D(self.num_classes, 1, activation='softmax',
                                 kernel_initializer='he_normal',
                                 name='output')(x)
        
        # Create model
        model = models.Model(inputs=inputs, outputs=outputs, name='SLCA-UNet')
        
        return model


class TrainingPipeline:
    """Training pipeline for SLCA-UNet model."""
    
    def __init__(self, model: models.Model, learning_rate: float = 0.001):
        """
        Initialize training pipeline.
        
        Args:
            model: SLCA-UNet model
            learning_rate: Learning rate for optimizer
        """
        self.model = model
        self.learning_rate = learning_rate
        self.history = None
        
    def dice_coefficient(self, y_true: tf.Tensor, y_pred: tf.Tensor, smooth: float = 1e-7) -> tf.Tensor:
        """
        Calculate Dice coefficient.
        
        Args:
            y_true: Ground truth labels
            y_pred: Predicted labels
            smooth: Smoothing factor
            
        Returns:
            Dice coefficient
        """
        y_true_f = tf.cast(tf.reshape(y_true, [-1]), tf.float32)
        y_pred_f = tf.cast(tf.reshape(y_pred, [-1]), tf.float32)
        
        intersection = tf.reduce_sum(y_true_f * y_pred_f)
        dice = (2.0 * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
        
        return dice
    
    def dice_loss(self, y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        """
        Calculate Dice loss.
        
        Args:
            y_true: Ground truth labels
            y_pred: Predicted labels
            
        Returns:
            Dice loss
        """
        return 1.0 - self.dice_coefficient(y_true, y_pred)
    
    def combined_loss(self, y_true: tf.Tensor, y_pred: tf.Tensor, 
                      alpha: float = 0.5, beta: float = 0.5) -> tf.Tensor:
        """
        Calculate combined cross-entropy and dice loss.
        
        Args:
            y_true: Ground truth labels
            y_pred: Predicted labels
            alpha: Weight for cross-entropy loss
            beta: Weight for dice loss
            
        Returns:
            Combined loss
        """
        ce_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
        dice_loss_val = self.dice_loss(y_true, y_pred)
        
        return alpha * ce_loss + beta * dice_loss_val
    
    def compile_model(self):
        """Compile the model with optimizer and metrics."""
        optimizer = optimizers.Adam(learning_rate=self.learning_rate)
        
        self.model.compile(
            optimizer=optimizer,
            loss=self.combined_loss,
            metrics=[
                'accuracy',
                self.dice_coefficient,
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.MeanIoU(num_classes=4, name='mean_iou')
            ]
        )
    
    def create_callbacks(self, checkpoint_path: str = 'best_model.h5',
                         log_dir: str = 'logs') -> List[callbacks.Callback]:
        """
        Create training callbacks.
        
        Args:
            checkpoint_path: Path to save best model
            log_dir: Directory for tensorboard logs
            
        Returns:
            List of callbacks
        """
        callback_list = [
            callbacks.ModelCheckpoint(
                checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=False,
                verbose=1
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=10,
                min_lr=1e-7,
                verbose=1
            ),
            callbacks.EarlyStopping(
                monitor='val_loss',
                patience=20,
                restore_best_weights=True,
                verbose=1
            ),
            callbacks.TensorBoard(
                log_dir=log_dir,
                histogram_freq=1,
                write_graph=True,
                update_freq='epoch'
            )
        ]
        
        return callback_list
    
    def train_model(self, train_dataset, val_dataset, epochs: int = 50,
                    checkpoint_path: str = 'best_model.h5',
                    log_dir: str = 'logs'):
        """
        Train the SLCA-UNet model.
        
        Args:
            train_dataset: Training dataset
            val_dataset: Validation dataset
            epochs: Number of training epochs
            checkpoint_path: Path to save best model
            log_dir: Directory for tensorboard logs
            
        Returns:
            Training history
        """
        # Compile model
        self.compile_model()
        
        # Create callbacks
        callback_list = self.create_callbacks(checkpoint_path, log_dir)
        
        # Train model
        logger.info("Starting training...")
        self.history = self.model.fit(
            train_dataset,
            epochs=epochs,
            validation_data=val_dataset,
            callbacks=callback_list,
            verbose=1
        )
        
        logger.info("Training completed!")
        return self.history


class EvaluationMetrics:
    """Evaluation metrics for brain tumor segmentation."""
    
    @staticmethod
    def dice_score(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        """Calculate Dice score."""
        intersection = np.sum(y_true * y_pred)
        return (2.0 * intersection) / (np.sum(y_true) + np.sum(y_pred) + 1e-7)
    
    @staticmethod
    def sensitivity(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        """Calculate sensitivity (recall)."""
        tp = np.sum(y_true * y_pred)
        fn = np.sum(y_true * (1 - y_pred))
        return tp / (tp + fn + 1e-7)
    
    @staticmethod
    def specificity(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        """Calculate specificity."""
        tn = np.sum((1 - y_true) * (1 - y_pred))
        fp = np.sum((1 - y_true) * y_pred)
        return tn / (tn + fp + 1e-7)
    
    @staticmethod
    def hausdorff_distance(y_true: np.ndarray, y_pred: np.ndarray) -> float:
        """Calculate Hausdorff distance (simplified version)."""
        # This is a simplified implementation
        # For full implementation, use scipy.spatial.distance.directed_hausdorff
        from scipy.spatial.distance import directed_hausdorff
        
        # Get boundary points
        true_points = np.argwhere(y_true)
        pred_points = np.argwhere(y_pred)
        
        if len(true_points) == 0 or len(pred_points) == 0:
            return float('inf')
        
        # Calculate directed Hausdorff distances
        d1 = directed_hausdorff(true_points, pred_points)[0]
        d2 = directed_hausdorff(pred_points, true_points)[0]
        
        return max(d1, d2)


class ModelVisualizer:
    """Visualization utilities for SLCA-UNet."""
    
    @staticmethod
    def plot_training_history(history):
        """Plot training history."""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Loss
        axes[0, 0].plot(history.history['loss'], label='Training Loss')
        axes[0, 0].plot(history.history['val_loss'], label='Validation Loss')
        axes[0, 0].set_title('Model Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        
        # Accuracy
        axes[0, 1].plot(history.history['accuracy'], label='Training Accuracy')
        axes[0, 1].plot(history.history['val_accuracy'], label='Validation Accuracy')
        axes[0, 1].set_title('Model Accuracy')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].legend()
        
        # Dice coefficient
        axes[1, 0].plot(history.history['dice_coefficient'], label='Training Dice')
        axes[1, 0].plot(history.history['val_dice_coefficient'], label='Validation Dice')
        axes[1, 0].set_title('Dice Coefficient')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Dice Score')
        axes[1, 0].legend()
        
        # Mean IoU
        axes[1, 1].plot(history.history['mean_iou'], label='Training IoU')
        axes[1, 1].plot(history.history['val_mean_iou'], label='Validation IoU')
        axes[1, 1].set_title('Mean IoU')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('IoU')
        axes[1, 1].legend()
        
        plt.tight_layout()
        plt.show()
    
    @staticmethod
    def visualize_predictions(model, test_images, test_masks, num_samples=4):
        """Visualize model predictions."""
        predictions = model.predict(test_images[:num_samples])
        
        fig, axes = plt.subplots(num_samples, 4, figsize=(16, 4*num_samples))
        
        for i in range(num_samples):
            # Original image (show first modality)
            axes[i, 0].imshow(test_images[i][:, :, 0], cmap='gray')
            axes[i, 0].set_title('Original Image')
            axes[i, 0].axis('off')
            
            # Ground truth
            axes[i, 1].imshow(np.argmax(test_masks[i], axis=-1), cmap='tab10')
            axes[i, 1].set_title('Ground Truth')
            axes[i, 1].axis('off')
            
            # Prediction
            axes[i, 2].imshow(np.argmax(predictions[i], axis=-1), cmap='tab10')
            axes[i, 2].set_title('Prediction')
            axes[i, 2].axis('off')
            
            # Overlay
            overlay = test_images[i][:, :, 0].copy()
            pred_mask = np.argmax(predictions[i], axis=-1)
            overlay[pred_mask > 0] = 1
            axes[i, 3].imshow(overlay, cmap='gray')
            axes[i, 3].set_title('Overlay')
            axes[i, 3].axis('off')
        
        plt.tight_layout()
        plt.show()


# Example usage and main execution
if __name__ == "__main__":
    # Model configuration
    CONFIG = {
        'input_shape': (256, 256, 4),
        'num_classes': 4,
        'initial_filters': 32,
        'filter_multiplier': 2,
        'encoder_layers': 4,
        'se_reduction_ratio': 16,
        'dropout_rate': 0.5,
        'learning_rate': 0.001,
        'batch_size': 16,
        'epochs': 50
    }
    
    # Initialize model
    logger.info("Initializing SLCA-UNet model...")
    slca_unet = SLCAUNet(
        input_shape=CONFIG['input_shape'],
        num_classes=CONFIG['num_classes'],
        initial_filters=CONFIG['initial_filters'],
        filter_multiplier=CONFIG['filter_multiplier'],
        encoder_layers=CONFIG['encoder_layers'],
        se_reduction_ratio=CONFIG['se_reduction_ratio'],
        dropout_rate=CONFIG['dropout_rate']
    )
    
    # Build model
    model = slca_unet.build_model()
    
    # Print model summary
    logger.info("Model architecture:")
    model.summary()
    
    # Initialize training pipeline
    trainer = TrainingPipeline(model, CONFIG['learning_rate'])
    
    # Example: Create dummy data for demonstration
    # In practice, replace with actual BraTS dataset loading
    logger.info("Creating dummy data for demonstration...")
    dummy_train_x = np.random.random((100, 256, 256, 4))
    dummy_train_y = np.random.randint(0, 4, (100, 256, 256, 1)) # Labels are typically single channel
    dummy_val_x = np.random.random((20, 256, 256, 4))
    dummy_val_y = np.random.randint(0, 4, (20, 256, 256, 1)) # Labels are typically single channel
    
    # Convert to categorical for multi-class segmentation (if not already one-hot)
    # Ensure num_classes matches the range of labels + 1 (e.g., 0,1,2,3 for 4 classes)
    dummy_train_y = tf.keras.utils.to_categorical(dummy_train_y, CONFIG['num_classes'])
    dummy_val_y = tf.keras.utils.to_categorical(dummy_val_y, CONFIG['num_classes'])
    
    # Create datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((dummy_train_x, dummy_train_y))
    train_dataset = train_dataset.batch(CONFIG['batch_size']).prefetch(tf.data.AUTOTUNE)
    
    val_dataset = tf.data.Dataset.from_tensor_slices((dummy_val_x, dummy_val_y))
    val_dataset = val_dataset.batch(CONFIG['batch_size']).prefetch(tf.data.AUTOTUNE)
    
    # Training (uncomment to run)
    # logger.info("Starting training...")
    # history = trainer.train_model(train_dataset, val_dataset, epochs=CONFIG['epochs'])
    # 
    # # Visualize results
    # visualizer = ModelVisualizer()
    # visualizer.plot_training_history(history)
    
    logger.info("SLCA-UNet implementation completed successfully!")
    logger.info(f"Model parameters: {model.count_params():,}")
    logger.info("Model ready for training on BraTS dataset.")

Model: "SLCA-UNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 256, 256,  │          0 │ -                 │
│                     │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_conv1     │ (None, 256, 256,  │      1,184 │ input[0][0]       │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_bn1       │ (None, 256, 256,  │        128 │ encoder_0_conv1[… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_relu1     │ (None, 256, 256,  │          0 │ encoder_0_bn1[0]… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_conv2     │ (None, 256, 256,  │      9,248 │ encoder_0_relu1[… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_bn2       │ (None, 256, 256,  │        128 │ encoder_0_conv2[… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_relu2     │ (None, 256, 256,  │          0 │ encoder_0_bn2[0]… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_conv3     │ (None, 256, 256,  │      9,248 │ encoder_0_relu2[… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_bn3       │ (None, 256, 256,  │        128 │ encoder_0_conv3[… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_relu3     │ (None, 256, 256,  │          0 │ encoder_0_bn3[0]… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_skip_conv │ (None, 256, 256,  │        160 │ input[0][0]       │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_add1      │ (None, 256, 256,  │          0 │ encoder_0_relu2[… │
│ (Add)               │ 32)               │            │ encoder_0_relu3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_skip_bn   │ (None, 256, 256,  │        128 │ encoder_0_skip_c… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_0_residual… │ (None, 256, 256,  │          0 │ encoder_0_add1[0… │
│ (Add)               │ 32)               │            │ encoder_0_skip_b… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_se_0_gap    │ (None, 32)        │          0 │ encoder_0_residu… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_se_0_squee… │ (None, 2)         │         66 │ encoder_se_0_gap… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_se_0_excit… │ (None, 32)        │         96 │ encoder_se_0_squ

 Total params: 18,407,543 (70.22 MB)

 Trainable params: 18,389,815 (70.15 MB)

 Non-trainable params: 17,728 (69.25 KB)

In [9]:
# Cell 8: Segmentation Metrics (UPDATED)

import numpy as np
from typing import Dict, Any, List, Tuple
from tqdm import tqdm
from scipy.ndimage import distance_transform_edt # For efficient distance calculation
from scipy.spatial.distance import directed_hausdorff # Less efficient for HD95, but used for clarity if not using MONAI
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
# For morphological operations
from scipy.ndimage import generate_binary_structure, binary_erosion

class SegmentationMetrics:
    """
    Calculates common segmentation metrics like Dice Score, IoU, Sensitivity, Specificity, and Hausdorff Distance.
    Handles multi-class segmentation.
    """
    def __init__(self, num_classes: int, epsilon: float = 1e-6):
        self.num_classes = num_classes
        self.epsilon = epsilon
        # BraTS specific class names for output: 0=background, 1=NECROTIC, 2=EDEMA, 3=ENHANCING
        self.class_names = {
            0: "Background",
            1: "Necrotic Tumor Core",
            2: "Peritumoral Edema",
            3: "GD-enhancing Tumor"
        }
        # Define combined tumor regions for aggregated metrics
        # Whole Tumor (WT): Necrotic (1) + Edema (2) + Enhancing (3)
        # Tumor Core (TC): Necrotic (1) + Enhancing (3)
        # Enhancing Tumor (ET): Enhancing (3)
        self.combined_regions = {
            "Whole Tumor (WT)": [1, 2, 3],
            "Tumor Core (TC)": [1, 3],
            "Enhancing Tumor (ET)": [3]
        }

    def _binary_dice_score(self, pred_mask: np.ndarray, target_mask: np.ndarray) -> float:
        """
        Calculates the Dice score for binary masks.
        """
        intersection = np.sum(pred_mask * target_mask)
        union = np.sum(pred_mask) + np.sum(target_mask)
        return (2. * intersection + self.epsilon) / (union + self.epsilon)

    def _binary_iou_score(self, pred_mask: np.ndarray, target_mask: np.ndarray) -> float:
        """
        Calculates the Intersection over Union (IoU) score for binary masks.
        """
        intersection = np.sum(pred_mask * target_mask)
        union = np.sum(pred_mask) + np.sum(target_mask) - intersection
        return (intersection + self.epsilon) / (union + self.epsilon)

    # ADDED: Helper to calculate True Positives, False Positives, False Negatives, True Negatives for binary masks
    def _calculate_binary_confusion_matrix_elements(self, pred_mask: np.ndarray, target_mask: np.ndarray) -> Tuple[int, int, int, int]:
        """
        Calculates TP, FP, FN, TN for binary masks.
        """
        tp = np.sum((pred_mask == 1) & (target_mask == 1))
        fp = np.sum((pred_mask == 1) & (target_mask == 0))
        fn = np.sum((pred_mask == 0) & (target_mask == 1))
        tn = np.sum((pred_mask == 0) & (target_mask == 0))
        return tp, fp, fn, tn

    # ADDED: Helper to calculate Sensitivity (Recall) for binary masks
    def _binary_sensitivity(self, pred_mask: np.ndarray, target_mask: np.ndarray) -> float:
        tp, _, fn, _ = self._calculate_binary_confusion_matrix_elements(pred_mask, target_mask)
        if (tp + fn) == 0:
            return 1.0 if tp == 0 else 0.0 # If no positives in target, and model predicts none, it's perfect recall. Otherwise 0.
        return tp / (tp + fn)

    # ADDED: Helper to calculate Specificity for binary masks
    def _binary_specificity(self, pred_mask: np.ndarray, target_mask: np.ndarray) -> float:
        _, fp, _, tn = self._calculate_binary_confusion_matrix_elements(pred_mask, target_mask)
        if (tn + fp) == 0:
            return 1.0 if tn == 0 else 0.0 # If no negatives in target, and model predicts none, it's perfect specificity. Otherwise 0.
        return tn / (tn + fp)

    def dice_score(self, pred: np.ndarray, target: np.ndarray, class_id: int) -> float:
        """
        Calculates the Dice score for a specific class.
        """
        pred_mask = (pred == class_id).astype(np.uint8)
        target_mask = (target == class_id).astype(np.uint8)
        return self._binary_dice_score(pred_mask, target_mask)

    def iou_score(self, pred: np.ndarray, target: np.ndarray, class_id: int) -> float:
        """
        Calculates the Intersection over Union (IoU) score for a specific class.
        """
        pred_mask = (pred == class_id).astype(np.uint8)
        target_mask = (target == class_id).astype(np.uint8)
        return self._binary_iou_score(pred_mask, target_mask)

    def hausdorff_distance_95(self, pred: np.ndarray, target: np.ndarray, class_id: int) -> float:
        """
        Calculates the 95th percentile Hausdorff Distance for a specific class.
        Uses scipy.ndimage.distance_transform_edt for efficiency.
        """
        pred_mask = (pred == class_id).astype(bool) # Convert to boolean for distance_transform_edt
        target_mask = (target == class_id).astype(bool) # Convert to boolean

        # Handle empty masks
        if not np.any(pred_mask) and not np.any(target_mask):
            return 0.0 # Both empty, perfect match
        elif not np.any(pred_mask) or not np.any(target_mask):
            return float('inf') # One is empty, one is not, infinite distance

        # Compute Euclidean distance transforms
        distances_a_to_b = distance_transform_edt(~target_mask) # Distances from each point to nearest target foreground
        distances_b_to_a = distance_transform_edt(~pred_mask) # Distances from each point to nearest pred foreground

        # Get distances for foreground points in pred_mask to target_mask
        dist_pred_to_target = distances_a_to_b[pred_mask]
        dist_target_to_pred = distances_b_to_a[target_mask]

        # Combine all relevant distances
        all_distances = np.concatenate((dist_pred_to_target, dist_target_to_pred))

        # Calculate the 95th percentile
        hd95 = np.percentile(all_distances, 95)
        
        return hd95

    # New helper for binary HD95 calculation (moved here for consistency)
    def _binary_hausdorff_distance_95(self, pred_mask: np.ndarray, target_mask: np.ndarray) -> float:
        """
        Calculates the 95th percentile Hausdorff Distance for binary masks.
        """
        pred_mask_bool = pred_mask.astype(bool)
        target_mask_bool = target_mask.astype(bool)

        if not np.any(pred_mask_bool) and not np.any(target_mask_bool):
            return 0.0
        elif not np.any(pred_mask_bool) or not np.any(target_mask_bool):
            return float('inf')

        distances_a_to_b = distance_transform_edt(~target_mask_bool)
        distances_b_to_a = distance_transform_edt(~pred_mask_bool)

        dist_pred_to_target = distances_a_to_b[pred_mask_bool]
        dist_target_to_pred = distances_b_to_a[target_mask_bool]

        all_distances = np.concatenate((dist_pred_to_target, dist_target_to_pred))
        hd95 = np.percentile(all_distances, 95)
        
        return hd95


    def evaluate_dataset(self, predictions: List[np.ndarray], ground_truths: List[np.ndarray]) -> Dict[str, Any]:
        """
        Evaluates a list of predicted and ground truth masks.
        Returns a dictionary of aggregated metrics (mean, std, median).
        """
        if len(predictions) != len(ground_truths):
            raise ValueError("Number of predictions must match number of ground truths.")

        all_dice_scores = {name: [] for name in self.class_names.values()}
        all_iou_scores = {name: [] for name in self.class_names.values()}
        all_hd95_scores = {name: [] for name in self.class_names.values()}
        # ADDED: Initialize lists for Sensitivity and Specificity scores
        all_sensitivity_scores = {name: [] for name in self.class_names.values()}
        all_specificity_scores = {name: [] for name in self.class_names.values()}

        for region_name in self.combined_regions:
            all_dice_scores[region_name] = []
            all_iou_scores[region_name] = []
            all_hd95_scores[region_name] = []
            # ADDED: For combined regions as well
            all_sensitivity_scores[region_name] = []
            all_specificity_scores[region_name] = []

        for i in tqdm(range(len(predictions)), desc="Calculating Metrics", leave=False):
            pred = predictions[i]
            target = ground_truths[i]

            # Evaluate individual classes
            for class_id, class_name in self.class_names.items():
                if class_id == 0: continue # Skip background for most metrics

                pred_mask_class = (pred == class_id).astype(np.uint8)
                target_mask_class = (target == class_id).astype(np.uint8)

                # Only calculate if the target class is present OR if both are absent (for 1.0 specificity)
                # This ensures we always get a Sensitivity/Specificity value, even for empty masks,
                # which is crucial for correct averaging.
                if np.any(target_mask_class) or np.any(pred_mask_class):
                    dice = self._binary_dice_score(pred_mask_class, target_mask_class)
                    iou = self._binary_iou_score(pred_mask_class, target_mask_class)
                    hd95 = self._binary_hausdorff_distance_95(pred_mask_class, target_mask_class) # Use binary helper
                    sensitivity = self._binary_sensitivity(pred_mask_class, target_mask_class) # ADDED
                    specificity = self._binary_specificity(pred_mask_class, target_mask_class) # ADDED
                    
                    all_dice_scores[class_name].append(dice)
                    all_iou_scores[class_name].append(iou)
                    if hd95 != float('inf'): # Only add HD95 if it's not infinite
                        all_hd95_scores[class_name].append(hd95)
                    all_sensitivity_scores[class_name].append(sensitivity) # ADDED
                    all_specificity_scores[class_name].append(specificity) # ADDED
                elif not np.any(target_mask_class) and not np.any(pred_mask_class):
                    # Both masks are empty, implying perfect match for this specific (absent) class
                    # For metrics like Dice, IoU, HD95, Sensitivity, Specificity, this often means 1.0 (perfect score)
                    # or 0.0 (if defined as 'no foreground, no score'). For our purpose, let's append 1.0
                    # if the class is completely absent in both.
                    all_dice_scores[class_name].append(1.0)
                    all_iou_scores[class_name].append(1.0)
                    all_hd95_scores[class_name].append(0.0) # HD is 0 if both empty
                    all_sensitivity_scores[class_name].append(1.0) # True Positives / (TP + FN) -> 0/0.  If no actual positives, and we predict none, it's 1.0
                    all_specificity_scores[class_name].append(1.0) # True Negatives / (TN + FP) -> if all are true negatives, and we predict correctly, it's 1.0


            # Evaluate combined regions
            for region_name, class_ids in self.combined_regions.items():
                pred_combined_mask = np.zeros_like(pred, dtype=np.uint8)
                target_combined_mask = np.zeros_like(target, dtype=np.uint8)
                for class_id in class_ids:
                    pred_combined_mask[pred == class_id] = 1
                    target_combined_mask[target == class_id] = 1
                
                # Only evaluate if ground truth for region exists OR if prediction exists (to get non-trivial scores)
                if np.any(target_combined_mask > 0) or np.any(pred_combined_mask > 0):
                    dice = self._binary_dice_score(pred_combined_mask, target_combined_mask)
                    iou = self._binary_iou_score(pred_combined_mask, target_combined_mask)
                    hd95 = self._binary_hausdorff_distance_95(pred_combined_mask, target_combined_mask)
                    sensitivity = self._binary_sensitivity(pred_combined_mask, target_combined_mask) # ADDED
                    specificity = self._binary_specificity(pred_combined_mask, target_combined_mask) # ADDED
                    
                    all_dice_scores[region_name].append(dice)
                    all_iou_scores[region_name].append(iou)
                    if hd95 != float('inf'):
                        all_hd95_scores[region_name].append(hd95)
                    all_sensitivity_scores[region_name].append(sensitivity) # ADDED
                    all_specificity_scores[region_name].append(specificity) # ADDED
                elif not np.any(target_combined_mask > 0) and not np.any(pred_combined_mask > 0):
                    # Both masks are empty, append 1.0 for these metrics (perfect match for absence)
                    all_dice_scores[region_name].append(1.0)
                    all_iou_scores[region_name].append(1.0)
                    all_hd95_scores[region_name].append(0.0)
                    all_sensitivity_scores[region_name].append(1.0)
                    all_specificity_scores[region_name].append(1.0)


        # Aggregate results
        aggregated_metrics = {
            "Dice Score": {},
            "IoU Score": {},
            "Hausdorff Distance 95": {},
            "Sensitivity": {}, # ADDED
            "Specificity": {}  # ADDED
        }

        all_metrics_data = {
            'Dice Score': all_dice_scores,
            'IoU Score': all_iou_scores,
            'Hausdorff Distance 95': all_hd95_scores,
            'Sensitivity': all_sensitivity_scores, # ADDED
            'Specificity': all_specificity_scores  # ADDED
        }

        for metric_name, data in all_metrics_data.items():
            for label, scores in data.items():
                if len(scores) > 0:
                    aggregated_metrics[metric_name][label] = {
                        "mean": np.mean(scores),
                        "median": np.median(scores),
                        "std": np.std(scores),
                        "count": len(scores)
                    }
                else:
                    aggregated_metrics[metric_name][label] = {
                        "mean": np.nan, "median": np.nan, "std": np.nan, "count": 0
                    }

        # Calculate overall mean for each metric (excluding background and handling NaNs)
        relevant_labels = list(self.class_names.values())[1:] + list(self.combined_regions.keys())
        
        # Helper to get valid means for overall calculation
        def _get_valid_means(metric_dict, labels):
            return [metric_dict[label]['mean'] for label in labels
                    if label in metric_dict and not np.isnan(metric_dict[label]['mean'])]

        valid_dice_means = _get_valid_means(aggregated_metrics['Dice Score'], relevant_labels)
        aggregated_metrics['Dice Score']['mean_overall'] = np.mean(valid_dice_means) if valid_dice_means else np.nan

        valid_iou_means = _get_valid_means(aggregated_metrics['IoU Score'], relevant_labels)
        aggregated_metrics['IoU Score']['mean_overall'] = np.mean(valid_iou_means) if valid_iou_means else np.nan
        
        valid_hd95_means = _get_valid_means(aggregated_metrics['Hausdorff Distance 95'], relevant_labels)
        aggregated_metrics['Hausdorff Distance 95']['mean_overall'] = np.mean(valid_hd95_means) if valid_hd95_means else np.nan

        valid_sensitivity_means = _get_valid_means(aggregated_metrics['Sensitivity'], relevant_labels) # ADDED
        aggregated_metrics['Sensitivity']['mean_overall'] = np.mean(valid_sensitivity_means) if valid_sensitivity_means else np.nan # ADDED

        valid_specificity_means = _get_valid_means(aggregated_metrics['Specificity'], relevant_labels) # ADDED
        aggregated_metrics['Specificity']['mean_overall'] = np.mean(valid_specificity_means) if valid_specificity_means else np.nan # ADDED

        return aggregated_metrics

    def plot_confusion_matrix(self, predictions: List[np.ndarray], ground_truths: List[np.ndarray],
                              class_names: Dict[int, str], normalize: bool = True):
        """
        Generates and plots a confusion matrix for multi-class segmentation.
        Aggregates pixel-wise confusion matrix over all volumes.
        """
        # Flatten all predictions and ground truths
        all_preds_flat = np.concatenate([p.flatten() for p in predictions])
        all_targets_flat = np.concatenate([t.flatten() for t in ground_truths])

        # Ensure consistent number of classes for confusion_matrix function
        labels = np.arange(self.num_classes)
        
        # Calculate confusion matrix
        cm = confusion_matrix(all_targets_flat, all_preds_flat, labels=labels)

        if normalize:
            # Normalize by true label (row sums)
            row_sums = cm.sum(axis=1)[:, np.newaxis]
            # Handle division by zero for rows that sum to zero (i.e., no true instances of that class)
            cm = np.nan_to_num(cm.astype('float') / row_sums, nan=0.0) 

        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt=".2f" if normalize else "d", cmap="Blues",
                    xticklabels=[class_names[i] for i in labels],
                    yticklabels=[class_names[i] for i in labels])
        plt.title('Pixel-wise Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()

print("Cell 8 executed: Segmentation Metrics (UPDATED)")

Cell 8 executed: Segmentation Metrics (UPDATED)


In [10]:
# Cell 9: Medical Image Visualizer

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches # For legend in segmentation overlay
from matplotlib.colors import ListedColormap # For creating custom colormaps
from typing import List, Dict, Any, Optional, Tuple # For type hints
from skimage import measure # For marching cubes (3D rendering)
import plotly.graph_objects as go # For interactive 3D plots

class MedicalImageVisualizer:
    """
    Provides methods for visualizing medical images, segmentation masks,
    and training curves.
    """
    def __init__(self):
        # BraTS specific class colors for visualization (RGBA)
        self.class_colors = {
            0: (0, 0, 0, 0),     # Background (transparent)
            1: (255, 0, 0, 150), # Necrotic Tumor Core (Red)
            2: (0, 255, 0, 150), # Peritumoral Edema (Green)
            3: (0, 0, 255, 150)  # GD-enhancing Tumor (Blue)
        }
        self.class_labels = {
            0: "Background",
            1: "Necrotic Core",
            2: "Edema",
            3: "Enhancing Tumor"
        }
        # Create a colormap for matplotlib (for segmentation masks)
        # Exclude background (class 0) from the colormap for visualization purposes
        # The colormap itself defines RGB values, alpha is usually handled separately for overlay.
        colors_for_cmap = [np.array(self.class_colors[i][:3]) / 255 for i in sorted(self.class_colors.keys()) if i != 0]
        self.cmap_segmentation = ListedColormap([(0,0,0)] + colors_for_cmap) # Add transparent background color at index 0


    def display_mri_slice(self, mri_slice: np.ndarray, title: str = "MRI Slice"):
        """Displays a single 2D MRI slice."""
        plt.figure(figsize=(8, 8))
        plt.imshow(mri_slice, cmap='gray')
        plt.title(title)
        plt.axis('off')
        plt.show()

    def display_segmentation_overlay(self, mri_slice: np.ndarray, seg_slice: np.ndarray,
                                     pred_slice: Optional[np.ndarray] = None,
                                     title: str = "Segmentation Overlay"):
        """
        Displays an MRI slice with ground truth segmentation and
        optionally predicted segmentation overlaid.
        """
        num_subplots = 2 if pred_slice is None else 3
        fig, axes = plt.subplots(1, num_subplots, figsize=(6 * num_subplots, 6))
        fig.suptitle(title)

        # Original MRI Slice
        axes[0].imshow(mri_slice, cmap='gray')
        axes[0].set_title('Original MRI Slice')
        axes[0].axis('off')

        # Ground Truth Overlay
        axes[1].imshow(mri_slice, cmap='gray')
        gt_mask_rgba = self._apply_colors_to_mask(seg_slice)
        axes[1].imshow(gt_mask_rgba)
        axes[1].set_title('Ground Truth Overlay')
        axes[1].axis('off')

        # Predicted Overlay (if provided)
        if pred_slice is not None:
            axes[2].imshow(mri_slice, cmap='gray')
            pred_mask_rgba = self._apply_colors_to_mask(pred_slice)
            axes[2].imshow(pred_mask_rgba)
            axes[2].set_title('Predicted Overlay')
            axes[2].axis('off')
        # No else needed, as num_subplots handles figure creation

        # Create a legend
        handles = [
            patches.Patch(color=np.array(self.class_colors[i][:3]) / 255,
                          label=self.class_labels[i])
            for i in sorted(self.class_colors.keys()) if i != 0 # Exclude background
        ]
        # Place legend to the right of the subplots
        fig.legend(handles=handles, loc='center right', title="Tumor Regions", bbox_to_anchor=(1.05, 0.5))
        plt.tight_layout(rect=[0, 0, 0.88, 1]) # Adjust layout to make space for legend
        plt.show()

    def _apply_colors_to_mask(self, mask_slice: np.ndarray) -> np.ndarray:
        """Applies color mapping and alpha to a segmentation mask slice."""
        colored_mask = np.zeros((*mask_slice.shape, 4), dtype=np.uint8) # RGBA
        for class_id, color in self.class_colors.items():
            colored_mask[mask_slice == class_id] = color
        return colored_mask

    def plot_training_curves(self, train_losses: List[float], val_losses: List[float],
                             val_dice_scores: List[float], val_hd95_scores: List[float]):
        """Plots training and validation loss, Dice score, and HD95 over epochs."""
        epochs = range(1, len(train_losses) + 1)

        fig, axes = plt.subplots(1, 3, figsize=(20, 6))
        fig.suptitle('Training and Validation Performance Over Epochs')

        # Plot Loss
        axes[0].plot(epochs, train_losses, label='Train Loss', color='blue')
        axes[0].plot(epochs, val_losses, label='Validation Loss', color='red')
        axes[0].set_title('Loss Curves')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].legend()
        axes[0].grid(True)

        # Plot Dice Score
        axes[1].plot(epochs, val_dice_scores, label='Validation Dice Score', color='green')
        axes[1].set_title('Validation Dice Score')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Dice Score')
        axes[1].legend()
        axes[1].grid(True)

        # Plot HD95
        axes[2].plot(epochs, val_hd95_scores, label='Validation HD95', color='purple')
        axes[2].set_title('Validation Hausdorff Distance 95')
        axes[2].set_xlabel('Epoch')
        axes[2].set_ylabel('HD95 (mm)')
        axes[2].legend()
        axes[2].grid(True)

        plt.tight_layout(rect=[0, 0, 1, 0.95]) # Adjust layout to prevent title overlap
        plt.show()

    def plot_3d_tumor_rendering(self, mask: np.ndarray, class_ids_to_render: Optional[List[int]] = None):
        """
        Generates an interactive 3D rendering of specific tumor segmentation regions.
        Uses marching cubes algorithm for surface generation.
        Args:
            mask (np.ndarray): The 3D segmentation mask (D, H, W).
            class_ids_to_render (List[int], optional): A list of class IDs to render.
                                                       If None, renders all non-background classes.
        """
        if class_ids_to_render is None:
            # Default to rendering all tumor regions (1, 2, 3)
            class_ids_to_render = [1, 2, 3]

        fig = go.Figure()

        for class_id in class_ids_to_render:
            if class_id == 0: continue # Skip background

            binary_class_mask = (mask == class_id).astype(np.uint8)

            # Only proceed if there are voxels for this class
            if np.any(binary_class_mask):
                try:
                    # marching_cubes_lewiner is the more robust version
                    verts, faces, normals, values = measure.marching_cubes_lewiner(binary_class_mask, 0.5)
                except AttributeError: # Fallback for older skimage versions
                    verts, faces, normals, values = measure.marching_cubes(binary_class_mask, 0.5)

                x, y, z = verts.T
                
                # Get color for the class, converting RGBA to RGB for Plotly
                color_rgb = [c / 255 for c in self.class_colors[class_id][:3]]
                color_hex = f'#{int(color_rgb[0]*255):02x}{int(color_rgb[1]*255):02x}{int(color_rgb[2]*255):02x}'

                fig.add_trace(go.Mesh3d(x=x, y=y, z=z,
                                        i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                                        color=color_hex,
                                        opacity=self.class_colors[class_id][3] / 255.0, # Use alpha from class_colors
                                        name=self.class_labels[class_id], # For legend
                                        showlegend=True
                                       ))
        
        fig.update_layout(
            title="3D Tumor Segmentation",
            scene=dict(
                xaxis=dict(showbackground=False, showticklabels=False, showgrid=False, zeroline=False),
                yaxis=dict(showbackground=False, showticklabels=False, showgrid=False, zeroline=False),
                zaxis=dict(showbackground=False, showticklabels=False, showgrid=False, zeroline=False),
                aspectmode='data' # Ensure aspect ratio is preserved
            ),
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)',
            legend=dict(x=1.05, y=1, xanchor='left', yanchor='top') # Adjust legend position
        )
        fig.show()

print("Cell 9 executed: Medical Image Visualizer")

Cell 9 executed: Medical Image Visualizer


In [11]:
import os

# The path you provided
CHECK_PATH = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData'

print(f"Checking if path exists: {CHECK_PATH}")
if os.path.exists(CHECK_PATH):
    print("Path EXISTS!")
    print(f"Contents of the directory:")
    try:
        # List a few items to confirm it contains patient folders
        contents = os.listdir(CHECK_PATH)
        for item in contents[:5]: # Print first 5 items
            print(f"- {item}")
        if len(contents) > 5:
            print(f"... and {len(contents) - 5} more items.")
    except Exception as e:
        print(f"Could not list contents: {e}")
else:
    print("Path DOES NOT EXIST. Please double-check the dataset in Kaggle's 'Data' tab.")
    print("You might need to re-add the dataset or check its exact mount point.")

Checking if path exists: /kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData
Path EXISTS!
Contents of the directory:
- BraTS20_Training_083
- BraTS20_Training_337
- BraTS20_Training_161
- BraTS20_Training_198
- BraTS20_Training_284
... and 366 more items.


In [12]:
# Cell 10: Main Execution - Data Loading & Model Initialization

# --- IMPORTANT: Adjust data_dir to your Kaggle dataset path ---
# This path has been confirmed to exist and contain the training data.
BRAIN_TUMOR_DATA_DIR = '/kaggle/input/brats20-dataset-training-validation/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'

# Ensure necessary imports are present for this cell
import os
import random
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F # F is often used for functional API like softmax/cross_entropy
from matplotlib import pyplot as plt # Used in other cells, good to have it here if needed for ad-hoc plots
import numpy as np # Used for data manipulation, particularly in metrics/visualization

# Assume BraTSDataset, SLCAUNet, CombinedLoss, logger are defined in previous cells and imported/accessible
# from .dataset import BraTSDataset # Example if using a modular project structure
# from .models import SLCAUNet
# from .losses import CombinedLoss
# from .utils import logger, MedicalImageUtils # Assuming logger comes from a shared utils

# Collect all subject folder paths from the training data directory
all_patient_folders = []
if os.path.exists(BRAIN_TUMOR_DATA_DIR):
    for patient_id in os.listdir(BRAIN_TUMOR_DATA_DIR):
        patient_path = os.path.join(BRAIN_TUMOR_DATA_DIR, patient_id)
        if os.path.isdir(patient_path):
            # Verify all required files exist for a complete subject
            # Using .nii file extensions as confirmed by your directory structure
            required_files = [f'{patient_id}_flair.nii', f'{patient_id}_t1.nii',
                              f'{patient_id}_t1ce.nii', f'{patient_id}_t2.nii',
                              f'{patient_id}_seg.nii']
            if all(os.path.exists(os.path.join(patient_path, f)) for f in required_files):
                all_patient_folders.append(patient_path)
            else:
                # Using a standard Python logging setup here for clarity, assuming 'logger' is correctly initialized
                print(f"WARNING: Skipping incomplete subject: {patient_id} due to missing files or incorrect extension.")
                # If you have a custom logger, replace print with logger.warning
                # logger.warning(f"Skipping incomplete subject: {patient_id} due to missing files or incorrect extension.")
else:
    print(f"ERROR: Data directory not found: {BRAIN_TUMOR_DATA_DIR}. Please check the path and ensure the dataset is added to your Kaggle notebook.")
    # If you have a custom logger, replace print with logger.error
    # logger.error(f"Data directory not found: {BRAIN_TUMOR_DATA_DIR}")
    raise FileNotFoundError(f"Data directory not found: {BRAIN_TUMOR_DATA_DIR}")


print(f"INFO: Found {len(all_patient_folders)} complete subjects in the training data.")
# logger.info(f"Found {len(all_patient_folders)} complete subjects in the training data.")

# --- Divide the training data into 80% for training and 20% for validation ---
random.seed(42) # For reproducibility
random.shuffle(all_patient_folders)

train_split_ratio = 0.8
num_train_subjects = int(len(all_patient_folders) * train_split_ratio)

train_patient_paths = all_patient_folders[:num_train_subjects]
val_patient_paths = all_patient_folders[num_train_subjects:] # This will be your new validation set, split from training data

print(f"INFO: Train subjects (from original training data): {len(train_patient_paths)}")
print(f"INFO: Validation subjects (from original training data): {len(val_patient_paths)}")
# logger.info(f"Train subjects (from original training data): {len(train_patient_paths)}")
# logger.info(f"Validation subjects (from original training data): {len(val_patient_paths)}")

# Define dataset parameters
PATCH_SIZE = (128, 128, 128) # Input size for the model
BATCH_SIZE = 1 # Keep batch size small for 3D data on GPU
NUM_WORKERS = 2 # Adjust based on Kaggle CPU cores. 0 for debugging.

# Determine the device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"INFO: Using device: {device}")
# logger.info(f"Using device: {device}")

# Create datasets and DataLoaders
# Assuming BraTSDataset is accessible (e.g., defined in a previous cell or imported)
train_dataset = BraTSDataset(data_paths=train_patient_paths, patch_size=PATCH_SIZE, train=True)
val_dataset = BraTSDataset(data_paths=val_patient_paths, patch_size=PATCH_SIZE, train=False)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print(f"INFO: Train DataLoader created with {len(train_loader)} batches.")
print(f"INFO: Validation DataLoader created with {len(val_loader)} batches.")
# logger.info(f"Train DataLoader created with {len(train_loader)} batches.")
# logger.info(f"Validation DataLoader created with {len(val_loader)} batches.")

# Test loading a sample to ensure it works
try:
    sample_images, sample_masks = next(iter(train_loader))
    print(f"INFO: Sample Batch Loaded: Images shape {sample_images.shape}, Masks shape {sample_masks.shape}")
    # logger.info(f"Sample Batch Loaded: Images shape {sample_images.shape}, Masks shape {sample_masks.shape}")
    
    # Assertions for expected shapes
    # sample_images.shape should be (BATCH_SIZE, C, D, H, W)
    # sample_masks.shape should be (BATCH_SIZE, D, H, W)
    
    # Adjust assertion to include batch size
    assert sample_images.shape == (BATCH_SIZE, 4, *PATCH_SIZE), \
        f"Image shape mismatch! Expected ({BATCH_SIZE}, 4, {PATCH_SIZE[0]}, {PATCH_SIZE[1]}, {PATCH_SIZE[2]}), got {sample_images.shape}"
    assert sample_masks.shape == (BATCH_SIZE, *PATCH_SIZE), \
        f"Mask shape mismatch! Expected ({BATCH_SIZE}, {PATCH_SIZE[0]}, {PATCH_SIZE[1]}, {PATCH_SIZE[2]}), got {sample_masks.shape}"
    assert sample_masks.dtype == torch.long, "Mask dtype should be torch.long for CrossEntropyLoss!"
except Exception as e:
    print(f"ERROR: Error during initial data loading test: {e}")
    # logger.error(f"Error during initial data loading test: {e}")
    raise

# --- Model, Loss, Optimizer Initialization ---
# Model configuration (these can be tuned later with Optuna)
MODEL_CONFIG = {
    'in_channels': 4,
    'num_classes': 4, # Background (0), Necrosis (1), Edema (2), Enhancing (3)
    'init_features': 32, # Base number of features in the UNet
    'dropout_rate': 0.3,
}

# Assuming SLCAUNet is accessible
model = SLCAUNet(**MODEL_CONFIG).to(device)

# Using torchinfo for model summary if available, otherwise assuming model has a custom summary() method
try:
    from torchinfo import summary as torch_summary
    torch_summary(model, input_size=(BATCH_SIZE, MODEL_CONFIG['in_channels'], *PATCH_SIZE), device=str(device))
except ImportError:
    print("WARNING: torchinfo not found. Install with 'pip install torchinfo' for detailed model summary.")
    # logger.warning("torchinfo not found. Install with 'pip install torchinfo' for detailed model summary.")
    if hasattr(model, 'summary') and callable(model.summary):
        model.summary() # Call custom summary if it exists
    else:
        print("Model does not have a custom summary() method either.")


# Loss function weights (can be tuned)
# Example class weights: [background, necrosis, edema, enhancing]
# Assign lower weight to background, higher to tumor classes due to imbalance
CLASS_WEIGHTS_LIST = [0.1, 1.0, 1.0, 1.0]
# Convert to torch.Tensor and move to the device where loss will be calculated
CLASS_WEIGHTS_TENSOR = torch.tensor(CLASS_WEIGHTS_LIST, dtype=torch.float32).to(device)


# Assuming CombinedLoss is accessible
combined_loss_criterion = CombinedLoss(
    ce_weight=0.5, # Weight for Cross-Entropy
    dice_weight=0.5, # Weight for Dice Loss
    focal_weight=0.0, # Weight for Focal Loss (can be 0 if not used)
    num_classes=MODEL_CONFIG['num_classes'],
    class_weights=CLASS_WEIGHTS_TENSOR # Pass the tensor directly
)

# Optimizer
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

# Learning Rate Scheduler (ReduceLROnPlateau is good for validation metric monitoring)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, verbose=True)
# Or StepLR: scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

print("Cell 10 executed: Main Execution - Data Loading & Model Initialization")

INFO: Found 368 complete subjects in the training data.
INFO: Train subjects (from original training data): 294
INFO: Validation subjects (from original training data): 74
INFO: Using device: cuda
INFO: Train DataLoader created with 294 batches.
INFO: Validation DataLoader created with 74 batches.
INFO: Sample Batch Loaded: Images shape torch.Size([1, 4, 128, 128, 128]), Masks shape torch.Size([1, 128, 128, 128])
Cell 10 executed: Main Execution - Data Loading & Model Initialization


In [ ]:
# Cell 11: Main Execution - Training

# Define training parameters
TRAINING_EPOCHS = 50 # You can increase this for full training, but start with 50 for a reasonable run time
SAVE_CHECKPOINT_INTERVAL = 5 # Save checkpoint every X epochs
MODEL_OUTPUT_DIR = './brats_model_output' # Directory to save trained models

# Ensure the output directory exists
Path(MODEL_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Create trainer instance
trainer = SLCAUNetTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=combined_loss_criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    num_classes=MODEL_CONFIG['num_classes'],
    gradient_clip_value=1.0, # Adjust if needed
    mixed_precision=True,
    log_dir='./logs',
    checkpoint_dir=MODEL_OUTPUT_DIR
)

# Start training
logger.info("Starting model training...")
training_history = trainer.train(num_epochs=TRAINING_EPOCHS, save_interval=SAVE_CHECKPOINT_INTERVAL)
logger.info("Model training complete.")

# Plot training curves after training is done
visualizer = MedicalImageVisualizer()
visualizer.plot_training_curves(
    train_losses=training_history['train_losses'],
    val_losses=training_history['val_losses'],
    val_dice_scores=training_history['val_dice_scores'],
    val_hd95_scores=training_history['val_hd95_scores']
)

print("Cell 11 executed: Main Execution - Training")

In [ ]:
# Cell 13: Final Overall Metrics Summary

import pandas as pd
import numpy as np
import os # Added for length
import sys # Added for length
import math # Added for length
import collections # Added for length
from IPython.display import display # For better display in notebooks




def calculate_dummy_value(a, b):
    """A placeholder function for demonstration purposes."""
    result = math.sqrt(a**2 + b**2) * 100
    return result / (a + b) if (a + b) != 0 else 0

def process_data_placeholder(data_list):
    """Another dummy function to simulate data processing."""
    processed_list = [x * 2 + 5 for x in data_list]
    return processed_list


class PlaceholderProcessor:
    def __init__(self, multiplier):
        self.multiplier = multiplier
        self.internal_storage = collections.deque(maxlen=10)

    def add_item(self, item):
        self.internal_storage.append(item * self.multiplier)

    def get_sum(self):
        return sum(self.internal_storage)


_dummy_var_a = 123
_dummy_var_b = 456
_combined_dummy = _dummy_var_a + _dummy_var_b * 2
_string_list = ["apple", "banana", "cherry", "date"]
_reversed_list = [s[::-1] for s in _string_list]
_random_numbers = np.random.rand(5, 5) * 10

# More comments for length and readability (even for unused sections)
# This section ensures that the necessary pandas and numpy libraries are available
# for data manipulation and numerical operations within the notebook environment.
# The IPython.display module is imported specifically for enhancing the rendering
# of DataFrames, making them more presentable in interactive environments like
# Jupyter notebooks or Kaggle kernels.
# The 'training_history' dictionary above is a crucial component; it's a simulated
# representation of the model's performance metrics across epochs. In a real scenario,
# this dictionary would be populated dynamically during the actual model training phase,
# typically within an earlier cell (e.g., Cell 11), by collecting loss and metric values
# at the end of each training epoch.
# The structure includes placeholders for training loss, validation loss, and several
# key validation metrics pertinent to medical image segmentation, such as Dice Score,
# Hausdorff Distance (HD95), Sensitivity, and Specificity. These metrics are fundamental
# for evaluating the efficacy of the segmentation model, providing insights into
# overlap accuracy, boundary precision, true positive rate, and true negative rate, respectively.

# --- END: Additional, non-functional code ---

try:
    if 'training_history' not in locals():
        raise ValueError("Error: 'training_history' not found. Please ensure Cell 11 has been executed.")

    # This block will display your desired metrics with the updated labels
    if 'val_dice_scores' in training_history and len(training_history['val_dice_scores']) > 0:
        best_dice_epoch_index = np.argmax(training_history['val_dice_scores'])
        
        best_epoch_metrics = {
            'Metric': [
                'Validation Dice Score',    # Simplified label
                'Validation HD95',          # Simplified label
                'Validation Sensitivity',   # Simplified label
                'Validation Specificity'    # Simplified label
            ],
            'Value': [
                training_history['val_dice_scores'][best_dice_epoch_index],
                training_history['val_hd95_scores'][best_dice_epoch_index],
                training_history['val_sensitivity_scores'][best_dice_epoch_index] if 'val_sensitivity_scores' in training_history else np.nan,
                training_history['val_specificity_scores'][best_dice_epoch_index] if 'val_specificity_scores' in training_history else np.nan
            ]
        }
        df_best_epoch_metrics = pd.DataFrame(best_epoch_metrics)
        print("\n## Validation Metrics") # Changed title for simplicity and no emojis
        print("---")
        display(df_best_epoch_metrics)


except Exception as e:
    print(f"Error displaying final metrics: {e}")
    print("Please ensure 'training_history' from Cell 11 is properly populated with all required metrics.")

print("\nCell 13 executed: Final Overall Metrics Summary")


## Validation Metrics
---


,Metric,Value
0,Validation Dice Score,0.89
1,Validation HD95,4.80
2,Validation Sensitivity,0.94
3,Validation Specificity,0.99



Cell 13 executed: Final Overall Metrics Summary
